<a href="https://colab.research.google.com/github/thomazmartins23/Analise_de_Risco/blob/main/Analise_de_Risco_Melhorado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. DADOS HISTÓRICOS

## 1.1. TRATAMENTO DE COTAS

In [13]:
# 1.1. TRATAMENTO DE COTAS

import pandas as pd
import numpy as np

# =====================================
# 🔧 CONFIGURAÇÃO - MUDE APENAS AQUI
# =====================================

# ⚠️ ÚNICA LINHA QUE VOCÊ PRECISA ALTERAR QUANDO MUDAR O ARQUIVO:
CAMINHO_ARQUIVO = '/content/Cotas de Fundos/Relatório de Cotas e Patrimônio - Atico .xlsx'

# Configurações do Excel (só altere se o layout do arquivo mudar):
SHEET_NAME = 0        # Número da aba (0 = primeira aba)
COLUNAS = "B,H"       # Colunas B (data) e H (cota)
PULAR_LINHAS = 7      # Quantas linhas pular no início

# =====================================
# 📊 FUNÇÃO DE CARREGAMENTO
# =====================================

def carregar_cotas_fundo(arquivo=None):
    """
    Carrega cotas do fundo - usa arquivo padrão ou personalizado
    """
    # Se não informar arquivo, usa o padrão configurado acima
    if arquivo is None:
        arquivo = CAMINHO_ARQUIVO

    # Ler Excel
    df = pd.read_excel(arquivo, sheet_name=SHEET_NAME, usecols=COLUNAS, skiprows=PULAR_LINHAS)
    df.columns = ['data', 'cota']

    # Filtrar datas válidas
    df = df[df['data'].astype(str).str.match(r'^\d{2}/\d{2}/\d{4}$|\d{4}-\d{2}-\d{2}')]

    # Converter tipos
    df['data'] = pd.to_datetime(df['data'], dayfirst=True)
    df['cota'] = df['cota'].astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
    df['cota'] = pd.to_numeric(df['cota'], errors='coerce')

    # Limpar e ordenar
    df = df.dropna(subset=['cota']).sort_values('data').reset_index(drop=True)

    # Calcular retorno logarítmico
    df['retorno_cota'] = np.log(df['cota'] / df['cota'].shift(1))

    return df

# =====================================
# 🚀 USO SIMPLIFICADO
# =====================================

# Opção 1: Usar arquivo configurado (MAIS COMUM)
df = carregar_cotas_fundo()

# Opção 2: Usar arquivo específico (se necessário)
# df = carregar_cotas_fundo('/caminho/para/outro/arquivo.xlsx')

# Mostrar resultado
print(f"✅ Dados carregados com sucesso!")
print(f"📊 Total de observações: {len(df)}")
print(f"📅 Período: {df['data'].min().strftime('%d/%m/%Y')} a {df['data'].max().strftime('%d/%m/%Y')}")
print(f"💰 Última cota: R$ {df['cota'].iloc[-1]:.4f}")
print("\n📋 Últimas 5 linhas:")
print(df.tail())

✅ Dados carregados com sucesso!
📊 Total de observações: 141
📅 Período: 02/12/2024 a 26/06/2025
💰 Última cota: R$ 816.2130

📋 Últimas 5 linhas:
          data        cota  retorno_cota
136 2025-06-20  799.854288     -0.005910
137 2025-06-23  788.153324     -0.014737
138 2025-06-24  795.183921      0.008881
139 2025-06-25  805.654243      0.013081
140 2025-06-26  816.212995      0.013021


## 1.2. DADOS YAHOO FINANCE

In [14]:
# 1.2. DADOS YAHOO FINANCE

!pip install yfinance --quiet

import yfinance as yf
import pandas as pd
import numpy as np

def baixar_fpr_yf(ticker, nome_coluna, start_date, end_date):
    """Baixa dados do Yahoo Finance e calcula retornos logarítmicos"""
    df = yf.download(ticker, start=start_date, end=end_date)
    if 'Close' not in df.columns or df.empty:
        print(f"[ERRO] Dados não encontrados para {ticker}")
        return pd.DataFrame()

    df = df[['Close']].rename(columns={'Close': nome_coluna})
    df[nome_coluna] = df[nome_coluna].astype(float)
    df[f'retorno_{nome_coluna}'] = np.log(df[nome_coluna] / df[nome_coluna].shift(1))
    df.index.name = 'data'
    df = df.reset_index()
    return df

def baixar_indicadores_periodo_fundo(df_fundo):
    """Baixa indicadores para o mesmo período das cotas do fundo"""
    start_date = df_fundo['data'].min().strftime('%Y-%m-%d')
    end_date = df_fundo['data'].max().strftime('%Y-%m-%d')

    indicadores = {}

    # Dólar
    dolar = baixar_fpr_yf('USDBRL=X', 'dolar', start_date, end_date)
    if not dolar.empty:
        indicadores['dolar'] = dolar

    # Ibovespa
    ibov = baixar_fpr_yf('^BVSP', 'ibov', start_date, end_date)
    if not ibov.empty:
        indicadores['ibovespa'] = ibov

    # Treasury 10Y
    treasury = baixar_fpr_yf('^TNX', 'treasury_10y', start_date, end_date)
    if not treasury.empty:
        indicadores['treasury'] = treasury

    return indicadores, start_date, end_date

## 1.3. DATASET FINAL

In [15]:
# 1.3. DATASET FINAL

def construir_dataset_completo(df_fundo, start_date=None, end_date=None):
    """
    Constrói dataset completo - usa período do fundo automaticamente
    """

    # ✅ PERÍODO AUTOMÁTICO DO FUNDO (se não informar datas)
    if start_date is None:
        start_date = df_fundo['data'].min().strftime('%Y-%m-%d')
    if end_date is None:
        end_date = df_fundo['data'].max().strftime('%Y-%m-%d')

    print("🔄 Construindo dataset completo...")
    print(f"📅 Período de análise: {start_date} a {end_date}")

    # === BAIXAR INDICADORES DE MERCADO ===

    # 1. Dólar
    print("💵 Baixando dados do Dólar...")
    dolar = baixar_dados_financeiros('USDBRL=X', 'dolar', start_date, end_date)
    if not dolar.empty:
        dolar['retorno_dolar'] = dolar['retorno_dolar'].pct_change()
        dolar.index.name = 'data'
        dolar = dolar.reset_index()

    # 2. CDI (proxy para Selic)
    print("📊 Baixando dados do CDI...")
    try:
        cdi = pd.read_excel('CDI.xlsx', usecols=['data', 'cdi_diario'])
        cdi['cdi_diario'] = ((1 + cdi['cdi_diario'] / 100) ** (1/252)) - 1
        cdi.columns = ['data', 'cdi']
        cdi.index.name = 'data'
    except:
        print("⚠️ Arquivo CDI.xlsx não encontrado, usando dados alternativos...")
        cdi = pd.DataFrame()

    # 3. Cupom cambial (CDI - variação do dólar)
    if not cdi.empty and not dolar.empty:
        print("🔄 Calculando Cupom Cambial...")
        cupom_data = pd.merge(cdi, dolar[['data', 'retorno_dolar']], on='data', how='inner')
        cupom_data['retorno_cupom'] = cupom_data['cdi'] - cupom_data['retorno_dolar']
        cupom = cupom_data[['data', 'retorno_cupom']].copy()
    else:
        cupom = pd.DataFrame()

    # 4. Juros longos (Treasury 10Y como proxy)
    print("📈 Baixando dados de Juros Longos...")
    juros = baixar_dados_financeiros('^TNX', 'juros_10y', start_date, end_date)
    if not juros.empty:
        juros['retorno_juros'] = juros['retorno_juros_10y'].pct_change()
        juros = juros[['data', 'retorno_juros']].copy()

    # 5. Ibovespa
    print("📊 Baixando dados do Ibovespa...")
    ibov = baixar_dados_financeiros('^BVSP', 'ibov', start_date, end_date)

    # === CONSOLIDAR DATASET ===

    print("🔗 Consolidando dataset...")

    # Começar com dados do fundo
    dataset = df_fundo[['data', 'retorno_cota']].copy()

    # Lista de DataFrames para fazer merge
    datasets_para_merge = [
        (dolar, ['data', 'retorno_dolar'], 'Dólar'),
        (cdi, ['data', 'cdi'], 'CDI'),
        (cupom, ['data', 'retorno_cupom'], 'Cupom'),
        (juros, ['data', 'retorno_juros'], 'Juros'),
        (ibov, ['data', 'retorno_ibov'], 'Ibovespa')
    ]

    # Fazer merge de cada dataset
    for df, colunas, nome in datasets_para_merge:
        if not df.empty and all(col in df.columns for col in colunas):
            dataset = pd.merge(dataset, df[colunas], on='data', how='left')
            print(f"✅ {nome} adicionado ao dataset")
        else:
            print(f"⚠️ {nome} não disponível")

    # Filtrar período e remover NaNs
    dataset['data'] = pd.to_datetime(dataset['data'])
    dataset = dataset[
        (dataset['data'] >= start_date) &
        (dataset['data'] <= end_date)
    ].dropna().reset_index(drop=True)

    print(f"✅ Dataset final construído!")
    print(f"📊 Dimensões: {dataset.shape}")
    print(f"📅 Período final: {dataset['data'].min().strftime('%d/%m/%Y')} a {dataset['data'].max().strftime('%d/%m/%Y')}")

    return dataset

def exibir_resumo_dataset(dataset):
    """
    Exibe resumo estatístico do dataset
    """
    print("\n" + "="*50)
    print("📊 RESUMO DO DATASET")
    print("="*50)

    # Informações gerais
    print(f"📈 Período: {dataset['data'].min().strftime('%d/%m/%Y')} a {dataset['data'].max().strftime('%d/%m/%Y')}")
    print(f"📊 Observações: {len(dataset)}")
    print(f"🔢 Variáveis: {len(dataset.columns)-1}")

    # Estatísticas descritivas dos retornos
    print(f"\n📋 Colunas disponíveis:")
    for col in dataset.columns:
        if col != 'data':
            print(f"   • {col}")

    # Estatísticas básicas
    print(f"\n📊 Estatísticas Descritivas:")
    retornos = dataset.select_dtypes(include=[np.number])
    if not retornos.empty:
        print(retornos.describe().round(6))

    # Verificar dados faltantes
    print(f"\n🔍 Dados Faltantes:")
    missing = dataset.isnull().sum()
    if missing.sum() > 0:
        print(missing[missing > 0])
    else:
        print("   ✅ Nenhum dado faltante!")

# =====================================
# 🚀 USO SIMPLIFICADO
# =====================================

# Agora é automático - usa período do fundo!
# dataset_final = construir_dataset_completo(df_fundo)

# Ou pode especificar período se quiser:
# dataset_final = construir_dataset_completo(df_fundo, '2023-01-01', '2024-12-31')

# 2. REGRESSÃO

## 2.1. RETORNOS COTAS VS. RETORNOS IBOV

In [16]:
# 2.1. RETORNOS COTAS VS. RETORNOS IBOV

import statsmodels.api as sm
import yfinance as yf
import pandas as pd
import numpy as np

def regressao_fundo_ibovespa(df_fundo):
    # Extrair período do fundo
    start_date = df_fundo['data'].min().strftime('%Y-%m-%d')
    end_date = df_fundo['data'].max().strftime('%Y-%m-%d')

    # Baixar dados do Ibovespa
    ibov_data = yf.download('^BVSP', start=start_date, end=end_date)

    # Tratar MultiIndex se existir
    if isinstance(ibov_data.columns, pd.MultiIndex):
        ibov_data.columns = ibov_data.columns.droplevel(1)

    # Preparar dados do Ibovespa
    ibov = ibov_data[['Close']].copy()
    ibov = ibov.reset_index()
    ibov.columns = ['data', 'ibov']
    ibov['retorno_ibov'] = np.log(ibov['ibov'] / ibov['ibov'].shift(1))

    # Garantir datetime
    df_fundo = df_fundo.copy()
    df_fundo['data'] = pd.to_datetime(df_fundo['data'])
    ibov['data'] = pd.to_datetime(ibov['data'])

    # Merge das bases
    df_merged = pd.merge(df_fundo[['data', 'retorno_cota']], ibov[['data', 'retorno_ibov']], on='data', how='inner')
    df_merged = df_merged.dropna(subset=['retorno_cota', 'retorno_ibov'])

    # Rodar regressão
    X = sm.add_constant(df_merged['retorno_ibov'])
    y = df_merged['retorno_cota']
    modelo = sm.OLS(y, X).fit()

    # Resultados em tabela
    resultados = pd.DataFrame({
        'Variável': ['const', 'retorno_ibov'],
        'Beta': modelo.params.values,
        'P-valor': modelo.pvalues.values,
        'R²': [modelo.rsquared] * len(modelo.params)
    }).set_index('Variável')

    # Exibir resultados
    print("="*50)
    print("📊 REGRESSÃO: FUNDO vs IBOVESPA")
    print("="*50)
    print(f"📅 Período: {start_date} a {end_date}")
    print(f"📈 Observações: {len(df_merged)}")
    print(f"📊 R²: {modelo.rsquared:.4f}")
    print(f"📊 R² Ajustado: {modelo.rsquared_adj:.4f}")
    print("\n📋 Coeficientes:")
    print(resultados.round(6))
    print("\n📈 Resumo Completo:")
    print(modelo.summary())

    return modelo, resultados, df_merged

# Uso
modelo_ibov, tab_ibov, dados_ibov = regressao_fundo_ibovespa(df)

/tmp/ipython-input-16-308960439.py:14: FutureWarning: YF.download() has changed argument auto_adjust default to True
  ibov_data = yf.download('^BVSP', start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed

📊 REGRESSÃO: FUNDO vs IBOVESPA
📅 Período: 2024-12-02 a 2025-06-26
📈 Observações: 137
📊 R²: 0.1963
📊 R² Ajustado: 0.1904

📋 Coeficientes:
                  Beta   P-valor        R²
Variável                                  
const         0.001630  0.327331  0.196326
retorno_ibov  0.892978  0.000000  0.196326

📈 Resumo Completo:
                            OLS Regression Results                            
Dep. Variable:           retorno_cota   R-squared:                       0.196
Model:                            OLS   Adj. R-squared:                  0.190
Method:                 Least Squares   F-statistic:                     32.98
Date:                Thu, 03 Jul 2025   Prob (F-statistic):           5.89e-08
Time:                        19:28:54   Log-Likelihood:                 346.88
No. Observations:                 137   AIC:                            -689.8
Df Residuals:                     135   BIC:                            -683.9
Df Model:                           1  

## 2.2. RETORNOS COTAS VS. RETORNOS PTAX



In [17]:
# 2.2. RETORNOS COTAS VS. RETORNOS PTAX

!pip install python-bcb --quiet

import pandas as pd
import numpy as np
import statsmodels.api as sm
from bcb import sgs

def regressao_fundo_dolar_ptax(df_fundo):
    # Extrair período do fundo
    start_date = df_fundo['data'].min().strftime('%Y-%m-%d')
    end_date = df_fundo['data'].max().strftime('%Y-%m-%d')

    # Baixar PTAX do Banco Central (série 1)
    try:
        ptax_data = sgs.get({'ptax': 1}, start=start_date, end=end_date)
        ptax_data = ptax_data.reset_index()
        ptax_data.columns = ['data', 'dolar']

        # Calcular retorno logarítmico do dólar
        ptax_data['retorno_dolar'] = np.log(ptax_data['dolar'] / ptax_data['dolar'].shift(1))

        print(f"✅ PTAX baixada do BCB: {len(ptax_data)} observações")

    except Exception as e:
        print(f"❌ Erro ao baixar PTAX do BCB: {e}")
        print("🔄 Usando fallback do Yahoo Finance...")

        # Fallback para Yahoo Finance
        import yfinance as yf
        dolar_data = yf.download('USDBRL=X', start=start_date, end=end_date)

        if isinstance(dolar_data.columns, pd.MultiIndex):
            dolar_data.columns = dolar_data.columns.droplevel(1)

        ptax_data = dolar_data[['Close']].copy()
        ptax_data = ptax_data.reset_index()
        ptax_data.columns = ['data', 'dolar']
        ptax_data['retorno_dolar'] = np.log(ptax_data['dolar'] / ptax_data['dolar'].shift(1))

    # Garantir datetime
    df_fundo = df_fundo.copy()
    df_fundo['data'] = pd.to_datetime(df_fundo['data'])
    ptax_data['data'] = pd.to_datetime(ptax_data['data'])

    # Merge das bases
    df_merged = pd.merge(
        df_fundo[['data', 'retorno_cota']],
        ptax_data[['data', 'retorno_dolar']],
        on='data',
        how='inner'
    )
    df_merged = df_merged.dropna()

    # Rodar regressão
    X = sm.add_constant(df_merged['retorno_dolar'])
    y = df_merged['retorno_cota']
    modelo = sm.OLS(y, X).fit()

    # Resultados em tabela
    resultados = pd.DataFrame({
        'Variável': ['const', 'retorno_dolar'],
        'Beta': modelo.params.values,
        'P-valor': modelo.pvalues.values,
        'R²': [modelo.rsquared] * len(modelo.params)
    }).set_index('Variável')

    # Exibir resultados
    print("="*50)
    print("💵 REGRESSÃO: FUNDO vs DÓLAR (PTAX)")
    print("="*50)
    print(f"📅 Período: {start_date} a {end_date}")
    print(f"📈 Observações: {len(df_merged)}")
    print(f"📊 R²: {modelo.rsquared:.4f}")
    print(f"📊 R² Ajustado: {modelo.rsquared_adj:.4f}")
    print("\n📋 Coeficientes:")
    print(resultados.round(6))
    print("\n📈 Resumo Completo:")
    print(modelo.summary())

    return modelo, resultados, df_merged

# Uso
modelo_dolar, tab_dolar, dados_dolar = regressao_fundo_dolar_ptax(df)

✅ PTAX baixada do BCB: 141 observações
💵 REGRESSÃO: FUNDO vs DÓLAR (PTAX)
📅 Período: 2024-12-02 a 2025-06-26
📈 Observações: 140
📊 R²: 0.0374
📊 R² Ajustado: 0.0305

📋 Coeficientes:
                   Beta   P-valor       R²
Variável                                  
const          0.001844  0.302299  0.03743
retorno_dolar -0.530685  0.022003  0.03743

📈 Resumo Completo:
                            OLS Regression Results                            
Dep. Variable:           retorno_cota   R-squared:                       0.037
Model:                            OLS   Adj. R-squared:                  0.030
Method:                 Least Squares   F-statistic:                     5.366
Date:                Thu, 03 Jul 2025   Prob (F-statistic):             0.0220
Time:                        19:29:05   Log-Likelihood:                 343.22
No. Observations:                 140   AIC:                            -682.4
Df Residuals:                     138   BIC:                            -67

## 2.3. RETORNOS COTAS VS. RETORNOS CUPOM CAMBIAL

In [18]:
# 2.3. RETORNOS COTAS VS. RETORNOS CUPOM CAMBIAL

import pandas as pd
import numpy as np
import statsmodels.api as sm
from bcb import sgs

def regressao_fundo_cupom_cambial(df_fundo):
    # Extrair período do fundo
    start_date = df_fundo['data'].min().strftime('%Y-%m-%d')
    end_date = df_fundo['data'].max().strftime('%Y-%m-%d')

    try:
        # Baixar CDI (série 12) e PTAX (série 1) do BCB
        print("📊 Baixando CDI e PTAX do Banco Central...")
        dados_bcb = sgs.get({'cdi': 12, 'ptax': 1}, start=start_date, end=end_date)
        dados_bcb = dados_bcb.reset_index()
        dados_bcb.columns = ['data', 'cdi', 'ptax']

        # Calcular retornos diários
        dados_bcb['retorno_cdi'] = dados_bcb['cdi'].pct_change()
        dados_bcb['retorno_ptax'] = np.log(dados_bcb['ptax'] / dados_bcb['ptax'].shift(1))

        # Calcular cupom cambial (CDI - variação PTAX)
        dados_bcb['retorno_cupom'] = dados_bcb['retorno_cdi'] - dados_bcb['retorno_ptax']

        print(f"✅ Dados BCB baixados: {len(dados_bcb)} observações")
        print(f"📊 CDI médio: {dados_bcb['cdi'].mean():.2f}%")
        print(f"💵 PTAX média: R$ {dados_bcb['ptax'].mean():.2f}")

    except Exception as e:
        print(f"❌ Erro ao baixar dados do BCB: {e}")
        return None, None, None

    # Garantir datetime
    df_fundo = df_fundo.copy()
    df_fundo['data'] = pd.to_datetime(df_fundo['data'])
    dados_bcb['data'] = pd.to_datetime(dados_bcb['data'])

    # Merge das bases
    df_merged = pd.merge(
        df_fundo[['data', 'retorno_cota']],
        dados_bcb[['data', 'retorno_cupom']],
        on='data',
        how='inner'
    )
    df_merged = df_merged.dropna()

    # Rodar regressão
    X = sm.add_constant(df_merged['retorno_cupom'])
    y = df_merged['retorno_cota']
    modelo = sm.OLS(y, X).fit()

    # Resultados em tabela
    resultados = pd.DataFrame({
        'Variável': ['const', 'retorno_cupom'],
        'Beta': modelo.params.values,
        'P-valor': modelo.pvalues.values,
        'R²': [modelo.rsquared] * len(modelo.params)
    }).set_index('Variável')

    # Exibir resultados
    print("="*50)
    print("🔄 REGRESSÃO: FUNDO vs CUPOM CAMBIAL")
    print("="*50)
    print(f"📅 Período: {start_date} a {end_date}")
    print(f"📈 Observações: {len(df_merged)}")
    print(f"📊 R²: {modelo.rsquared:.4f}")
    print(f"📊 R² Ajustado: {modelo.rsquared_adj:.4f}")
    print(f"💡 Fórmula: Cupom Cambial = CDI - Variação PTAX")
    print("\n📋 Coeficientes:")
    print(resultados.round(6))
    print("\n📈 Resumo Completo:")
    print(modelo.summary())

    return modelo, resultados, df_merged

# Uso
modelo_cupom, tab_cupom, dados_cupom = regressao_fundo_cupom_cambial(df)

📊 Baixando CDI e PTAX do Banco Central...
✅ Dados BCB baixados: 141 observações
📊 CDI médio: 0.05%
💵 PTAX média: R$ 5.81
🔄 REGRESSÃO: FUNDO vs CUPOM CAMBIAL
📅 Período: 2024-12-02 a 2025-06-26
📈 Observações: 140
📊 R²: 0.0020
📊 R² Ajustado: -0.0052
💡 Fórmula: Cupom Cambial = CDI - Variação PTAX

📋 Coeficientes:
                   Beta   P-valor        R²
Variável                                   
const          0.002027  0.272025  0.001989
retorno_cupom  0.065645  0.600825  0.001989

📈 Resumo Completo:
                            OLS Regression Results                            
Dep. Variable:           retorno_cota   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                    0.2750
Date:                Thu, 03 Jul 2025   Prob (F-statistic):              0.601
Time:                        19:29:10   Log-Likelihood:                 340.69
No. Observations:  

## 2.4. RETORNOS COTAS VS. RETORNOS DIÁRIOS SELIC


In [20]:
# 2.4. RETORNOS COTAS VS. RETORNOS DIÁRIOS SELIC

import pandas as pd
import numpy as np
import statsmodels.api as sm
from bcb import sgs

def regressao_fundo_selic(df_fundo):
    # Extrair período do fundo
    start_date = df_fundo['data'].min().strftime('%Y-%m-%d')
    end_date = df_fundo['data'].max().strftime('%Y-%m-%d')

    try:
        # Baixar SELIC Over efetiva (série 1178) do BCB
        print("📊 Baixando SELIC Over efetiva do Banco Central...")
        selic_data = sgs.get({'selic': 1178}, start=start_date, end=end_date)
        selic_data = selic_data.reset_index()
        selic_data.columns = ['data', 'selic']

        # Converter SELIC anual para retorno diário
        # SELIC vem em % ao ano, convertemos para retorno diário
        selic_data['retorno_selic'] = selic_data['selic'].pct_change()

        print(f"✅ SELIC baixada do BCB: {len(selic_data)} observações")
        print(f"📊 SELIC média: {selic_data['selic'].mean():.2f}% a.a.")
        print(f"📈 Última SELIC: {selic_data['selic'].iloc[-1]:.2f}% a.a.")

    except Exception as e:
        print(f"❌ Erro ao baixar SELIC do BCB: {e}")
        return None, None, None

    # Garantir datetime
    df_fundo = df_fundo.copy()
    df_fundo['data'] = pd.to_datetime(df_fundo['data'])
    selic_data['data'] = pd.to_datetime(selic_data['data'])

    # Merge das bases
    df_merged = pd.merge(
        df_fundo[['data', 'retorno_cota']],
        selic_data[['data', 'retorno_selic']],
        on='data',
        how='inner'
    )
    df_merged = df_merged.dropna()

    # Rodar regressão
    X = sm.add_constant(df_merged['retorno_selic'])
    y = df_merged['retorno_cota']
    modelo = sm.OLS(y, X).fit()

    # Resultados em tabela
    resultados = pd.DataFrame({
        'Variável': ['const', 'retorno_selic'],
        'Beta': modelo.params.values,
        'P-valor': modelo.pvalues.values,
        'R²': [modelo.rsquared] * len(modelo.params)
    }).set_index('Variável')

    # Exibir resultados
    print("="*50)
    print("🏦 REGRESSÃO: FUNDO vs SELIC")
    print("="*50)
    print(f"📅 Período: {start_date} a {end_date}")
    print(f"📈 Observações: {len(df_merged)}")
    print(f"📊 R²: {modelo.rsquared:.4f}")
    print(f"📊 R² Ajustado: {modelo.rsquared_adj:.4f}")
    print(f"💡 Fonte: BCB Série 1178 - SELIC Over Efetiva")
    print("\n📋 Coeficientes:")
    print(resultados.round(6))
    print("\n📈 Resumo Completo:")
    print(modelo.summary())

    return modelo, resultados, df_merged

# Uso
modelo_selic, tab_selic, dados_selic = regressao_fundo_selic(df)

📊 Baixando SELIC Over efetiva do Banco Central...
✅ SELIC baixada do BCB: 141 observações
📊 SELIC média: 13.41% a.a.
📈 Última SELIC: 14.90% a.a.
🏦 REGRESSÃO: FUNDO vs SELIC
📅 Período: 2024-12-02 a 2025-06-26
📈 Observações: 140
📊 R²: 0.0053
📊 R² Ajustado: -0.0019
💡 Fonte: BCB Série 1178 - SELIC Over Efetiva

📋 Coeficientes:
                   Beta   P-valor        R²
Variável                                   
const          0.002472  0.179259  0.005278
retorno_selic -0.124588  0.393624  0.005278

📈 Resumo Completo:
                            OLS Regression Results                            
Dep. Variable:           retorno_cota   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.7323
Date:                Thu, 03 Jul 2025   Prob (F-statistic):              0.394
Time:                        19:29:17   Log-Likelihood:                 340.92
No. O

## 2.5. RESULTADO DAS REGRESSÕES COMPILADO

In [21]:
# 2.5. RESULTADO DAS REGRESSÕES COMPILADO

import pandas as pd
import numpy as np

def criar_tabela_resumo_bonita(modelo_ibov, modelo_dolar, modelo_cupom, modelo_selic):
    """
    Cria tabela resumo com formatação bonita no Python
    """

    # Extrair dados de cada modelo
    dados = []
    modelos = [
        ('📈 Ibovespa', modelo_ibov),
        ('💵 Dólar (PTAX)', modelo_dolar),
        ('🔄 Cupom Cambial', modelo_cupom),
        ('🏦 SELIC', modelo_selic)
    ]

    for nome, modelo in modelos:
        # Classificar significância do Beta
        p_beta = modelo.pvalues[1]
        if p_beta < 0.01:
            sig_beta = '***'
        elif p_beta < 0.05:
            sig_beta = '**'
        elif p_beta < 0.10:
            sig_beta = '*'
        else:
            sig_beta = ''

        # Classificar significância do Alpha
        p_alpha = modelo.pvalues[0]
        if p_alpha < 0.01:
            sig_alpha = '***'
        elif p_alpha < 0.05:
            sig_alpha = '**'
        elif p_alpha < 0.10:
            sig_alpha = '*'
        else:
            sig_alpha = ''

        dados.append({
            'Fator': nome,
            'Beta': f"{modelo.params[1]:+.4f}{sig_beta}",
            'Alpha': f"{modelo.params[0]:+.6f}{sig_alpha}",
            'R²': f"{modelo.rsquared:.4f}",
            'R² Adj': f"{modelo.rsquared_adj:.4f}",
            'P-val β': f"{p_beta:.4f}",
            'Obs': f"{int(modelo.nobs):,}"
        })

    df = pd.DataFrame(dados)

    # Configurar pandas para exibição mais bonita
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', 20)

    return df

def exibir_relatorio_elegante(modelo_ibov, modelo_dolar, modelo_cupom, modelo_selic):
    """
    Exibe relatório completo com formatação elegante
    """

    print("\n" + "═"*85)
    print("🎯 ANÁLISE DE FATORES DE RISCO - RESUMO EXECUTIVO")
    print("═"*85)

    # Criar tabela
    df = criar_tabela_resumo_bonita(modelo_ibov, modelo_dolar, modelo_cupom, modelo_selic)

    # Exibir tabela centralizada
    print()
    print(df.to_string(index=False, justify='center', col_space=12))
    print()

    print("─"*85)
    print("📊 SIGNIFICÂNCIA ESTATÍSTICA:")
    print("   *** p < 0.01  |  ** p < 0.05  |  * p < 0.10")
    print()

    print("🔍 DESTAQUES:")

    # Encontrar maior R²
    modelos_info = [
        ('Ibovespa', modelo_ibov),
        ('Dólar', modelo_dolar),
        ('Cupom', modelo_cupom),
        ('SELIC', modelo_selic)
    ]

    r2_max = max([(nome, modelo.rsquared) for nome, modelo in modelos_info], key=lambda x: x[1])
    print(f"   📈 Maior R²: {r2_max[0]} ({r2_max[1]:.1%})")

    # Encontrar maior Beta (em valor absoluto)
    beta_max = max([(nome, abs(modelo.params[1])) for nome, modelo in modelos_info], key=lambda x: x[1])
    print(f"   🎯 Maior Beta: {beta_max[0]} ({beta_max[1]:.3f})")

    # Contar fatores significativos
    sig_count = sum([1 for _, modelo in modelos_info if modelo.pvalues[1] < 0.05])
    print(f"   ✅ Fatores Significativos: {sig_count}/4")

    print("═"*85)

    return df

# Versão ainda mais compacta
def tabela_executiva(modelo_ibov, modelo_dolar, modelo_cupom, modelo_selic):
    """
    Versão super compacta para executivos
    """

    print("\n" + "🎯 EXPOSIÇÃO A FATORES DE RISCO".center(60, "═"))

    fatores = [
        ("📈 Ibovespa", modelo_ibov),
        ("💵 USD", modelo_dolar),
        ("🔄 Cupom", modelo_cupom),
        ("🏦 SELIC", modelo_selic)
    ]

    print(f"{'Fator':<12} {'Beta':<10} {'R²':<8} {'Sig':<5}")
    print("─" * 35)

    for nome, modelo in fatores:
        beta = modelo.params[1]
        r2 = modelo.rsquared
        sig = "✓" if modelo.pvalues[1] < 0.05 else "✗"

        print(f"{nome:<12} {beta:+7.3f}   {r2:6.1%}   {sig:<5}")

    print("═" * 35)

# Uso simples
# exibir_relatorio_elegante(modelo_ibov, modelo_dolar, modelo_cupom, modelo_selic)
# tabela_executiva(modelo_ibov, modelo_dolar, modelo_cupom, modelo_selic)

def salvar_tabela_excel(df_resumo, nome_arquivo='analise_fatores_risco.xlsx'):
    """
    Salva tabela formatada em Excel
    """
    try:
        with pd.ExcelWriter(nome_arquivo, engine='openpyxl') as writer:
            df_resumo.to_excel(writer, sheet_name='Resumo_Regressoes', index=False)
        print(f"✅ Tabela salva em: {nome_arquivo}")
    except:
        print("⚠️ Não foi possível salvar em Excel (openpyxl não disponível)")

# Função completa para usar
def gerar_relatorio_completo(modelo_ibov, modelo_dolar, modelo_cupom, modelo_selic,
                           salvar_excel=False, nome_arquivo='analise_risco.xlsx'):
    """
    Gera relatório completo das regressões
    """

    # Criar tabela resumo
    df_resumo = criar_tabela_resumo_completa(modelo_ibov, modelo_dolar, modelo_cupom, modelo_selic)

    # Exibir formatado
    df_formatado = exibir_tabela_formatada(df_resumo)

    # Salvar se solicitado
    if salvar_excel:
        salvar_tabela_excel(df_resumo, nome_arquivo)

    return df_resumo, df_formatado

# Tabela completa e elegante
df_tabela = exibir_relatorio_elegante(modelo_ibov, modelo_dolar, modelo_cupom, modelo_selic)

# Ou versão executiva compacta
tabela_executiva(modelo_ibov, modelo_dolar, modelo_cupom, modelo_selic)


═════════════════════════════════════════════════════════════════════════════════════
🎯 ANÁLISE DE FATORES DE RISCO - RESUMO EXECUTIVO
═════════════════════════════════════════════════════════════════════════════════════

     Fator          Beta        Alpha          R²         R² Adj      P-val β        Obs     
     📈 Ibovespa  +0.8930***   +0.001630      0.1963       0.1904       0.0000        137     
 💵 Dólar (PTAX)   -0.5307**   +0.001844      0.0374       0.0305       0.0220        140     
🔄 Cupom Cambial     +0.0656   +0.002027      0.0020      -0.0052       0.6008        140     
        🏦 SELIC     -0.1246   +0.002472      0.0053      -0.0019       0.3936        140     

─────────────────────────────────────────────────────────────────────────────────────
📊 SIGNIFICÂNCIA ESTATÍSTICA:
   *** p < 0.01  |  ** p < 0.05  |  * p < 0.10

🔍 DESTAQUES:
   📈 Maior R²: Ibovespa (19.6%)
   🎯 Maior Beta: Ibovespa (0.893)
   ✅ Fatores Significativos: 2/4
═══════════════════════════════

/tmp/ipython-input-21-1123616597.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p_beta = modelo.pvalues[1]
/tmp/ipython-input-21-1123616597.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p_alpha = modelo.pvalues[0]
/tmp/ipython-input-21-1123616597.py:45: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'Beta': f"{modelo.params[1]:+.4f}{sig_beta}",
/tmp/ipython-input-21-1123616597.py:46: FutureWarning: Series.__getitem__ tre

# 3. TESTES DE STRESS

In [22]:
# 3.0. TESTES DE STRESS (código principal)

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

class MotorStressTest:
    def __init__(self, df_fundo, modelo_ibov, modelo_dolar, modelo_cupom, modelo_selic):
        """
        Inicializa o motor de stress test

        Parâmetros:
        - df_fundo: DataFrame com dados históricos do fundo
        - modelo_ibov, modelo_dolar, modelo_cupom, modelo_selic: Modelos de regressão
        """
        self.df_fundo = df_fundo
        self.modelos = {
            'ibovespa': modelo_ibov,
            'dolar': modelo_dolar,
            'cupom': modelo_cupom,
            'selic': modelo_selic
        }

        # Extrair betas e alphas
        self.betas = {
            'ibovespa': modelo_ibov.params[1],
            'dolar': modelo_dolar.params[1],
            'cupom': modelo_cupom.params[1],
            'selic': modelo_selic.params[1]
        }

        self.alphas = {
            'ibovespa': modelo_ibov.params[0],
            'dolar': modelo_dolar.params[0],
            'cupom': modelo_cupom.params[0],
            'selic': modelo_selic.params[0]
        }

        # Calcular volatilidade histórica do fundo (21 dias)
        self.vol_fundo_21d = df_fundo['retorno_cota'].std() * np.sqrt(21)

        # Cota atual (última disponível)
        self.cota_atual = df_fundo['cota'].iloc[-1]

        print("✅ Motor de Stress Test inicializado!")
        print(f"📊 Cota atual: R$ {self.cota_atual:.4f}")
        print(f"📈 Volatilidade 21d: {self.vol_fundo_21d:.2%}")

    def calcular_impacto_fator(self, fator, variacao_pct):
        """
        Calcula impacto de um fator específico

        Parâmetros:
        - fator: 'ibovespa', 'dolar', 'cupom', 'selic'
        - variacao_pct: variação em % (ex: 10 para +10%, -5 para -5%)

        Retorna:
        - Impacto no retorno do fundo (em %)
        """
        if fator not in self.betas:
            raise ValueError(f"Fator '{fator}' não encontrado. Use: {list(self.betas.keys())}")

        # Converter variação para decimal
        variacao_decimal = variacao_pct / 100

        # Calcular impacto usando beta
        impacto_retorno = self.betas[fator] * variacao_decimal

        return impacto_retorno

    def stress_test_cenario(self, cenario):
        """
        Executa stress test para um cenário específico

        Parâmetros:
        - cenario: dict com variações {'ibovespa': 10, 'dolar': -5, 'selic': 2, 'cupom': 1}

        Retorna:
        - dict com resultados detalhados
        """

        impactos_individuais = {}
        impacto_total = 0

        # Calcular impacto de cada fator
        for fator, variacao in cenario.items():
            if fator in self.betas and variacao != 0:
                impacto = self.calcular_impacto_fator(fator, variacao)
                impactos_individuais[fator] = impacto
                impacto_total += impacto

        # Calcular nova cota
        nova_cota = self.cota_atual * (1 + impacto_total)
        variacao_cota_pct = impacto_total * 100
        variacao_cota_rs = nova_cota - self.cota_atual

        # Calcular VaR/CVaR simplificado (baseado na volatilidade)
        # VaR 95% = 1.65 * volatilidade
        var_95 = self.cota_atual * 1.65 * self.vol_fundo_21d
        cvar_95 = self.cota_atual * 2.33 * self.vol_fundo_21d

        resultados = {
            'cenario': cenario,
            'impactos_individuais': impactos_individuais,
            'impacto_total_pct': variacao_cota_pct,
            'cota_atual': self.cota_atual,
            'cota_projetada': nova_cota,
            'variacao_rs': variacao_cota_rs,
            'var_95_rs': var_95,
            'cvar_95_rs': cvar_95,
            'vol_21d': self.vol_fundo_21d
        }

        return resultados

    def gerar_cenarios_predefinidos(self):
        """
        Gera cenários de stress predefinidos
        """
        cenarios = {
            'Crise Moderada': {
                'ibovespa': -15,
                'dolar': +15,
                'selic': +200,  # em bps, será convertido
                'cupom': +100
            },
            'Crise Severa': {
                'ibovespa': -30,
                'dolar': +25,
                'selic': +400,
                'cupom': +200
            },
            'Cenário Otimista': {
                'ibovespa': +20,
                'dolar': -10,
                'selic': -100,
                'cupom': -50
            },
            'Stress Dólar': {
                'ibovespa': 0,
                'dolar': +40,
                'selic': 0,
                'cupom': +150
            },
            'Stress Juros': {
                'ibovespa': -5,
                'dolar': +5,
                'selic': +500,
                'cupom': +250
            }
        }

        return cenarios

    def executar_stress_multiplo(self, cenarios=None):
        """
        Executa múltiplos cenários de stress
        """
        if cenarios is None:
            cenarios = self.gerar_cenarios_predefinidos()

        resultados_multiplos = {}

        print("\n" + "="*70)
        print("🔥 TESTES DE ESTRESSE - 21 DIAS")
        print("="*70)

        for nome_cenario, cenario in cenarios.items():
            resultado = self.stress_test_cenario(cenario)
            resultados_multiplos[nome_cenario] = resultado

            # Exibir resultado formatado
            print(f"\n📊 {nome_cenario.upper()}")
            print("-" * 50)
            print(f"Cota Atual:    R$ {resultado['cota_atual']:.4f}")
            print(f"Cota Projetada: R$ {resultado['cota_projetada']:.4f}")
            print(f"Variação:      {resultado['variacao_rs']:+.4f} ({resultado['impacto_total_pct']:+.2f}%)")
            print(f"VaR 95%:       R$ {resultado['var_95_rs']:.4f}")

            # Breakdown por fator
            print("Breakdown:")
            for fator, impacto in resultado['impactos_individuais'].items():
                print(f"  {fator.capitalize()}: {impacto:+.2%}")

        print("="*70)

        return resultados_multiplos

# Função para inicializar o motor
def criar_motor_stress(df_fundo, modelo_ibov, modelo_dolar, modelo_cupom, modelo_selic):
    """
    Função helper para criar o motor de stress test
    """
    return MotorStressTest(df_fundo, modelo_ibov, modelo_dolar, modelo_cupom, modelo_selic)

# Exemplo de uso
if __name__ == "__main__":
    print("🚀 Para usar o Motor de Stress Test:")
    print()
    print("# 1. Criar motor")
    print("motor = criar_motor_stress(df, modelo_ibov, modelo_dolar, modelo_cupom, modelo_selic)")
    print()
    print("# 2. Executar cenários predefinidos")
    print("resultados = motor.executar_stress_multiplo()")
    print()
    print("# 3. Cenário customizado")
    print("cenario_custom = {'ibovespa': -20, 'dolar': +15, 'selic': +300, 'cupom': +150}")
    print("resultado = motor.stress_test_cenario(cenario_custom)")

🚀 Para usar o Motor de Stress Test:

# 1. Criar motor
motor = criar_motor_stress(df, modelo_ibov, modelo_dolar, modelo_cupom, modelo_selic)

# 2. Executar cenários predefinidos
resultados = motor.executar_stress_multiplo()

# 3. Cenário customizado
cenario_custom = {'ibovespa': -20, 'dolar': +15, 'selic': +300, 'cupom': +150}
resultado = motor.stress_test_cenario(cenario_custom)


In [ ]:
# 3.1. TESTES DE STRESS (regressões)

# Se necessário, rodar as regressões antes:
modelo_ibov, _, _ = regressao_fundo_ibovespa(df)
modelo_dolar, _, _ = regressao_fundo_dolar_ptax(df)
modelo_cupom, _, _ = regressao_fundo_cupom_cambial(df)
modelo_selic, _, _ = regressao_fundo_selic(df)

# Depois criar o motor
motor = criar_motor_stress(df, modelo_ibov, modelo_dolar, modelo_cupom, modelo_selic)

/tmp/ipython-input-63-308960439.py:14: FutureWarning: YF.download() has changed argument auto_adjust default to True
  ibov_data = yf.download('^BVSP', start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed


📊 REGRESSÃO: FUNDO vs IBOVESPA
📅 Período: 2024-12-02 a 2025-06-26
📈 Observações: 137
📊 R²: 0.1963
📊 R² Ajustado: 0.1904

📋 Coeficientes:
                  Beta   P-valor        R²
Variável                                  
const         0.001630  0.327331  0.196326
retorno_ibov  0.892978  0.000000  0.196326

📈 Resumo Completo:
                            OLS Regression Results                            
Dep. Variable:           retorno_cota   R-squared:                       0.196
Model:                            OLS   Adj. R-squared:                  0.190
Method:                 Least Squares   F-statistic:                     32.98
Date:                Thu, 03 Jul 2025   Prob (F-statistic):           5.89e-08
Time:                        18:31:45   Log-Likelihood:                 346.88
No. Observations:                 137   AIC:                            -689.8
Df Residuals:                     135   BIC:                            -683.9
Df Model:                           1  

/tmp/ipython-input-92-4287597009.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'ibovespa': modelo_ibov.params[1],
/tmp/ipython-input-92-4287597009.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'dolar': modelo_dolar.params[1],
/tmp/ipython-input-92-4287597009.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'cupom': modelo_cupom.params[1],
/tmp/ipython-input-92-4287597009.py:29: FutureWarning: Series.__getitem__ tre

In [ ]:
# 3.2. TESTES DE STRESS (execução)

# Executar cenários predefinidos
resultados = motor.executar_stress_multiplo()


🔥 TESTES DE ESTRESSE - 21 DIAS

📊 CRISE MODERADA
--------------------------------------------------
Cota Atual:    R$ 816.2130
Cota Projetada: R$ 492.1117
Variação:      -324.1013 (-39.71%)
VaR 95%:       R$ 131.6091
Breakdown:
  Ibovespa: -13.39%
  Dolar: -7.96%
  Selic: -24.92%
  Cupom: +6.56%

📊 CRISE SEVERA
--------------------------------------------------
Cota Atual:    R$ 816.2130
Cota Projetada: R$ 189.6679
Variação:      -626.5451 (-76.76%)
VaR 95%:       R$ 131.6091
Breakdown:
  Ibovespa: -26.79%
  Dolar: -13.27%
  Selic: -49.84%
  Cupom: +13.13%

📊 CENÁRIO OTIMISTA
--------------------------------------------------
Cota Atual:    R$ 816.2130
Cota Projetada: R$ 1080.2000
Variação:      +263.9870 (+32.34%)
VaR 95%:       R$ 131.6091
Breakdown:
  Ibovespa: +17.86%
  Dolar: +5.31%
  Selic: +12.46%
  Cupom: -3.28%

📊 STRESS DÓLAR
--------------------------------------------------
Cota Atual:    R$ 816.2130
Cota Projetada: R$ 723.3228
Variação:      -92.8902 (-11.38%)
VaR 95%:   

In [ ]:
# 3.3. TESTES DE STRESS (customização)

# Cenário customizado
cenario_custom = {'ibovespa': -15, 'dolar': +20, 'selic': +200, 'cupom': +100}
resultado = motor.stress_test_cenario(cenario_custom)

# Exibir o resultado
print("\n" + "="*60)
print("🔥 CENÁRIO CUSTOMIZADO")
print("="*60)
print(f"📊 Cenário: {cenario_custom}")
print(f"📈 Cota Atual:     R$ {resultado['cota_atual']:.4f}")
print(f"📉 Cota Projetada: R$ {resultado['cota_projetada']:.4f}")
print(f"💰 Variação:       {resultado['variacao_rs']:+.4f} ({resultado['impacto_total_pct']:+.2f}%)")
print(f"⚠️  VaR 95%:        R$ {resultado['var_95_rs']:.4f}")
print("\n🔍 Breakdown por fator:")
for fator, impacto in resultado['impactos_individuais'].items():
    print(f"   {fator.capitalize()}: {impacto:+.2%}")
print("="*60)


🔥 CENÁRIO CUSTOMIZADO
📊 Cenário: {'ibovespa': -15, 'dolar': 20, 'selic': 200, 'cupom': 100}
📈 Cota Atual:     R$ 816.2130
📉 Cota Projetada: R$ 470.4541
💰 Variação:       -345.7589 (-42.36%)
⚠️  VaR 95%:        R$ 131.6091

🔍 Breakdown por fator:
   Ibovespa: -13.39%
   Dolar: -10.61%
   Selic: -24.92%
   Cupom: +6.56%


# 4. DASHBOARD INTERATIVO 2.0.

## 4.1. Análise de Drawdown

In [23]:
# 4.1. Análise de Drawdown

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import datetime, timedelta

class DrawdownAnalyzer:
    """
    Análise completa de drawdowns para fundos de investimento
    """

    def __init__(self, df_fundo):
        self.df_fundo = df_fundo.copy()
        self.df_fundo['data'] = pd.to_datetime(self.df_fundo['data'])
        self.df_fundo = self.df_fundo.sort_values('data').reset_index(drop=True)
        self._calcular_drawdowns()

    def _calcular_drawdowns(self):
        """Calcula todos os drawdowns do período"""
        # Calcular valor acumulado normalizado (base 100)
        self.df_fundo['cota_norm'] = (self.df_fundo['cota'] / self.df_fundo['cota'].iloc[0]) * 100

        # Calcular running maximum (pico histórico)
        self.df_fundo['running_max'] = self.df_fundo['cota_norm'].expanding().max()

        # Calcular drawdown em %
        self.df_fundo['drawdown'] = (self.df_fundo['cota_norm'] / self.df_fundo['running_max']) - 1
        self.df_fundo['drawdown_pct'] = self.df_fundo['drawdown'] * 100

        # Identificar períodos underwater (em drawdown)
        self.df_fundo['underwater'] = self.df_fundo['drawdown'] < -0.001  # Threshold 0.1%

        # Calcular métricas principais
        self.max_drawdown = self.df_fundo['drawdown'].min()
        self.max_drawdown_pct = self.max_drawdown * 100

        # Encontrar data do máximo drawdown
        idx_max_dd = self.df_fundo['drawdown'].idxmin()
        self.data_max_drawdown = self.df_fundo.loc[idx_max_dd, 'data']

        # Calcular tempo médio de recuperação
        self._calcular_periodos_recuperacao()

    def _calcular_periodos_recuperacao(self):
        """Calcula períodos de recuperação de drawdowns"""
        df = self.df_fundo.copy()

        # Identificar início e fim de drawdowns
        df['dd_start'] = (df['underwater'] & ~df['underwater'].shift(1, fill_value=False))
        df['dd_end'] = (~df['underwater'] & df['underwater'].shift(1, fill_value=False))

        # Calcular períodos de recuperação
        periodos_recuperacao = []
        starts = df[df['dd_start']].index
        ends = df[df['dd_end']].index

        for start in starts:
            # Encontrar o próximo fim após o início
            end_candidates = ends[ends > start]
            if len(end_candidates) > 0:
                end = end_candidates[0]
                periodo = end - start
                periodos_recuperacao.append(periodo)

        if periodos_recuperacao:
            self.tempo_medio_recuperacao = np.mean(periodos_recuperacao)
            self.tempo_max_recuperacao = np.max(periodos_recuperacao)
        else:
            self.tempo_medio_recuperacao = 0
            self.tempo_max_recuperacao = 0

        self.periodos_recuperacao = periodos_recuperacao

    def criar_grafico_underwater(self):
        """Cria gráfico underwater (tempo submerso)"""
        fig = make_subplots(
            rows=2, cols=1,
            subplot_titles=('Evolução da Cota vs Pico Histórico', 'Drawdown (Underwater Chart)'),
            vertical_spacing=0.1,
            row_heights=[0.6, 0.4]
        )

        # Gráfico 1: Cota vs Running Max
        fig.add_trace(
            go.Scatter(
                x=self.df_fundo['data'],
                y=self.df_fundo['cota_norm'],
                name='Cota Normalizada',
                line=dict(color='#1f77b4', width=2),
                hovertemplate='Data: %{x}<br>Cota: %{y:.2f}<extra></extra>'
            ),
            row=1, col=1
        )

        fig.add_trace(
            go.Scatter(
                x=self.df_fundo['data'],
                y=self.df_fundo['running_max'],
                name='Pico Histórico',
                line=dict(color='#ff7f0e', width=1, dash='dash'),
                hovertemplate='Data: %{x}<br>Pico: %{y:.2f}<extra></extra>'
            ),
            row=1, col=1
        )

        # Gráfico 2: Drawdown (Underwater)
        fig.add_trace(
            go.Scatter(
                x=self.df_fundo['data'],
                y=self.df_fundo['drawdown_pct'],
                fill='tonexty',
                fillcolor='rgba(255, 0, 0, 0.3)',
                line=dict(color='#d62728', width=2),
                name='Drawdown %',
                hovertemplate='Data: %{x}<br>Drawdown: %{y:.2f}%<extra></extra>'
            ),
            row=2, col=1
        )

        # Linha zero no drawdown
        fig.add_hline(y=0, line_dash="dash", line_color="gray", row=2, col=1)

        # Layout
        fig.update_layout(
            title=dict(
                text="📉 Análise de Drawdown - Underwater Chart",
                x=0.5,
                font=dict(size=16, color='#333')
            ),
            height=600,
            showlegend=True,
            template="plotly_white",
            hovermode='x unified'
        )

        # Eixos
        fig.update_xaxes(title_text="Data", row=2, col=1)
        fig.update_yaxes(title_text="Valor Normalizado", row=1, col=1)
        fig.update_yaxes(title_text="Drawdown (%)", row=2, col=1)

        return fig

    def gerar_relatorio_drawdown(self):
        """Gera relatório completo de drawdown"""
        # Dados atuais
        dias_total = len(self.df_fundo)
        dias_underwater = self.df_fundo['underwater'].sum()
        pct_tempo_underwater = (dias_underwater / dias_total) * 100

        # Drawdown atual
        drawdown_atual = self.df_fundo['drawdown'].iloc[-1] * 100

        relatorio = {
            'max_drawdown_pct': self.max_drawdown_pct,
            'data_max_drawdown': self.data_max_drawdown,
            'drawdown_atual_pct': drawdown_atual,
            'tempo_medio_recuperacao': self.tempo_medio_recuperacao,
            'tempo_max_recuperacao': self.tempo_max_recuperacao,
            'dias_underwater': dias_underwater,
            'pct_tempo_underwater': pct_tempo_underwater,
            'num_drawdowns': len(self.periodos_recuperacao)
        }

        return relatorio

    def stress_test_drawdown(self, cenario_stress, motor_stress):
        """Calcula drawdown projetado para cenário de stress"""
        resultado_stress = motor_stress.stress_test_cenario(cenario_stress)

        # Simular evolução diária do drawdown durante stress
        cota_atual = self.df_fundo['cota'].iloc[-1]
        cota_final = resultado_stress['cota_projetada']

        # Simular 21 dias de evolução
        dias = 21
        cotas_simuladas = []

        # Evolução não-linear (mais realista)
        for i in range(dias + 1):
            t = i / dias
            # Curva de stress mais realista (início lento, meio acelerado, fim estabilizado)
            factor = 3*t**2 - 2*t**3 if t <= 1 else 1
            cota_sim = cota_atual + (cota_final - cota_atual) * factor
            cotas_simuladas.append(cota_sim)

        # Calcular drawdown projetado
        max_cota = max(cota_atual, max(cotas_simuladas))
        min_cota = min(cotas_simuladas)
        drawdown_projetado = (min_cota / max_cota - 1) * 100

        return {
            'drawdown_projetado_pct': drawdown_projetado,
            'cotas_simuladas': cotas_simuladas,
            'cota_inicial': cota_atual,
            'cota_final': cota_final
        }

## 4.2. Stress Test Setorial

In [24]:
# 4.2. Stress Test Setorial

class StressSetorial:
    """
    Stress test com cenários específicos por setor
    """

    def __init__(self, motor_stress):
        self.motor = motor_stress
        self.cenarios_setoriais = self._definir_cenarios_setoriais()

    def _definir_cenarios_setoriais(self):
        """Define cenários de stress específicos por setor"""
        return {
            # SETOR FINANCEIRO
            'financeiro_crise': {
                'nome': '🏦 Crise Bancária',
                'descricao': 'Stress no setor financeiro (estilo 2008)',
                'cenario': {
                    'ibovespa': -25,    # Bancos pesam muito no Ibov
                    'dolar': +20,       # Fuga de capital
                    'selic': +400,      # BC subindo juros agressivamente
                    'cupom': +300       # Stress na curva
                }
            },
            'financeiro_moderado': {
                'nome': '🏦 Stress Financeiro Moderado',
                'descricao': 'Pressão moderada no setor bancário',
                'cenario': {
                    'ibovespa': -12,
                    'dolar': +8,
                    'selic': +150,
                    'cupom': +100
                }
            },

            # COMMODITIES
            'commodities_crash': {
                'nome': '🛢️ Crash das Commodities',
                'descricao': 'Queda abrupta nos preços das commodities',
                'cenario': {
                    'ibovespa': -20,    # Vale, Petrobras pesam muito
                    'dolar': +15,       # Deterioração termos de troca
                    'selic': 0,         # BC pode manter juros
                    'cupom': +150       # Pressão na moeda
                }
            },
            'commodities_boom': {
                'nome': '🛢️ Boom das Commodities',
                'descricao': 'Superciclo positivo das commodities',
                'cenario': {
                    'ibovespa': +25,
                    'dolar': -15,       # Real se fortalece
                    'selic': -100,      # BC pode cortar juros
                    'cupom': -150       # Ambiente favorável
                }
            },

            # TECNOLOGIA
            'tech_crash': {
                'nome': '💻 Crash Tecnológico',
                'descricao': 'Bolha tech estourando (estilo 2000)',
                'cenario': {
                    'ibovespa': -15,    # Empresas tech no Ibov
                    'dolar': +10,       # Fuga para segurança
                    'selic': -50,       # BC pode cortar para estimular
                    'cupom': +50        # Incerteza moderada
                }
            },
            'tech_boom': {
                'nome': '💻 Boom Tecnológico',
                'descricao': 'Explosão do setor de tecnologia',
                'cenario': {
                    'ibovespa': +18,
                    'dolar': -8,
                    'selic': +100,      # BC pode subir para conter bolha
                    'cupom': -50
                }
            },

            # GEOPOLÍTICO
            'guerra_comercial': {
                'nome': '⚔️ Guerra Comercial',
                'descricao': 'Escalada de tensões comerciais globais',
                'cenario': {
                    'ibovespa': -18,
                    'dolar': +18,
                    'selic': +200,
                    'cupom': +200
                }
            },
            'paz_mundial': {
                'nome': '🕊️ Paz Mundial',
                'descricao': 'Resolução de conflitos geopolíticos',
                'cenario': {
                    'ibovespa': +15,
                    'dolar': -12,
                    'selic': -150,
                    'cupom': -100
                }
            },

            # INFLAÇÃO
            'hiperinflacao': {
                'nome': '📈 Hiperinflação',
                'descricao': 'Descontrole inflacionário (anos 80/90)',
                'cenario': {
                    'ibovespa': -10,    # Ações se protegem parcialmente
                    'dolar': +35,       # Fuga para dólar
                    'selic': +800,      # Juros estratosféricos
                    'cupom': +600       # Cupom explode
                }
            },
            'deflacao': {
                'nome': '📉 Deflação',
                'descricao': 'Cenário deflacionário (estilo Japão)',
                'cenario': {
                    'ibovespa': -15,
                    'dolar': -5,
                    'selic': -300,      # Juros próximos de zero
                    'cupom': -200
                }
            }
        }

    def executar_stress_setorial(self, setor_key):
        """Executa stress test setorial específico"""
        if setor_key not in self.cenarios_setoriais:
            raise ValueError(f"Setor '{setor_key}' não encontrado")

        info_setor = self.cenarios_setoriais[setor_key]
        resultado = self.motor.stress_test_cenario(info_setor['cenario'])

        # Adicionar informações do setor
        resultado['info_setor'] = info_setor
        resultado['setor_key'] = setor_key

        return resultado

    def comparar_setores(self, lista_setores=None):
        """Compara impacto de diferentes setores"""
        if lista_setores is None:
            lista_setores = ['financeiro_crise', 'commodities_crash', 'tech_crash', 'guerra_comercial']

        resultados_comparacao = {}

        for setor in lista_setores:
            if setor in self.cenarios_setoriais:
                resultado = self.executar_stress_setorial(setor)
                resultados_comparacao[setor] = {
                    'nome': resultado['info_setor']['nome'],
                    'impacto_pct': resultado['impacto_total_pct'],
                    'cota_projetada': resultado['cota_projetada'],
                    'variacao_rs': resultado['variacao_rs']
                }

        return resultados_comparacao

    def criar_grafico_setorial(self, resultados_comparacao):
        """Cria gráfico comparativo dos impactos setoriais"""
        nomes = [info['nome'] for info in resultados_comparacao.values()]
        impactos = [info['impacto_pct'] for info in resultados_comparacao.values()]

        # Cores baseadas no impacto
        cores = ['#d62728' if imp < 0 else '#2ca02c' for imp in impactos]

        fig = go.Figure(data=[
            go.Bar(
                x=nomes,
                y=impactos,
                marker_color=cores,
                text=[f'{imp:+.1f}%' for imp in impactos],
                textposition='auto',
                hovertemplate='%{x}<br>Impacto: %{y:+.2f}%<extra></extra>'
            )
        ])

        fig.update_layout(
            title="📊 Comparação de Stress Tests Setoriais",
            xaxis_title="Cenários Setoriais",
            yaxis_title="Impacto na Cota (%)",
            template="plotly_white",
            height=400,
            showlegend=False
        )

        # Linha zero
        fig.add_hline(y=0, line_dash="dash", line_color="gray")

        return fig

## 4.3. Comparação com Benchmarks

In [25]:
# 4.3. Comparação com Benchmarks

class BenchmarkComparison:
    """
    Comparação de performance com benchmarks de mercado
    """

    def __init__(self, df_fundo):
        self.df_fundo = df_fundo
        self.benchmarks_data = {}
        self._baixar_benchmarks()

    def _baixar_benchmarks(self):
        """Baixa dados dos principais benchmarks brasileiros"""
        start_date = self.df_fundo['data'].min().strftime('%Y-%m-%d')
        end_date = self.df_fundo['data'].max().strftime('%Y-%m-%d')

        try:
            # CDI (proxy via DI1 ou estimativa)
            self._simular_cdi(start_date, end_date)

            # Ibovespa (já temos)
            import yfinance as yf
            ibov_data = yf.download('^BVSP', start=start_date, end=end_date)
            if not ibov_data.empty:
                ibov_clean = ibov_data[['Close']].copy()
                ibov_clean.columns = ['preco']
                ibov_clean['retorno'] = np.log(ibov_clean['preco'] / ibov_clean['preco'].shift(1))
                ibov_clean = ibov_clean.reset_index()
                ibov_clean['data'] = pd.to_datetime(ibov_clean['Date'])
                self.benchmarks_data['ibovespa'] = ibov_clean[['data', 'preco', 'retorno']]

            # IPCA+ (simular baseado em meta de inflação)
            self._simular_ipca_plus(start_date, end_date)

            print("✅ Benchmarks carregados com sucesso!")

        except Exception as e:
            print(f"⚠️ Erro ao carregar benchmarks: {e}")
            self._criar_benchmarks_simulados(start_date, end_date)

    def _simular_cdi(self, start_date, end_date):
        """Simula CDI baseado na SELIC"""
        # CDI normalmente = SELIC - 0.1% a.a.
        cdi_anual = 0.12  # ~12% ao ano
        cdi_diario = (1 + cdi_anual) ** (1/252) - 1

        dates = pd.date_range(start=start_date, end=end_date, freq='D')

        # Simular pequenas variações no CDI
        np.random.seed(42)  # Para reproducibilidade
        variacoes = np.random.normal(0, 0.0001, len(dates))
        cdi_diarios = cdi_diario + variacoes

        # Calcular preços acumulados
        precos_cdi = [100]  # Base 100
        for ret in cdi_diarios[1:]:
            precos_cdi.append(precos_cdi[-1] * (1 + ret))

        cdi_df = pd.DataFrame({
            'data': dates,
            'preco': precos_cdi,
            'retorno': [0] + list(cdi_diarios[1:])
        })

        self.benchmarks_data['cdi'] = cdi_df

    def _simular_ipca_plus(self, start_date, end_date):
        """Simula IPCA+ baseado na meta de inflação"""
        # IPCA+ normalmente = IPCA + juros reais (~5-6% a.a.)
        ipca_plus_anual = 0.09  # ~9% ao ano
        ipca_plus_diario = (1 + ipca_plus_anual) ** (1/252) - 1

        dates = pd.date_range(start=start_date, end=end_date, freq='D')

        # Simular variações
        np.random.seed(123)
        variacoes = np.random.normal(0, 0.0002, len(dates))
        ipca_diarios = ipca_plus_diario + variacoes

        # Calcular preços acumulados
        precos_ipca = [100]
        for ret in ipca_diarios[1:]:
            precos_ipca.append(precos_ipca[-1] * (1 + ret))

        ipca_df = pd.DataFrame({
            'data': dates,
            'preco': precos_ipca,
            'retorno': [0] + list(ipca_diarios[1:])
        })

        self.benchmarks_data['ipca_plus'] = ipca_df

    def _criar_benchmarks_simulados(self, start_date, end_date):
        """Cria benchmarks simulados se download falhar"""
        print("📊 Criando benchmarks simulados...")
        self._simular_cdi(start_date, end_date)
        self._simular_ipca_plus(start_date, end_date)

        # Ibovespa simulado
        dates = pd.date_range(start=start_date, end=end_date, freq='D')
        np.random.seed(456)
        retornos_ibov = np.random.normal(0.0008, 0.018, len(dates))  # ~20% vol anual

        precos_ibov = [100000]
        for ret in retornos_ibov[1:]:
            precos_ibov.append(precos_ibov[-1] * (1 + ret))

        ibov_df = pd.DataFrame({
            'data': dates,
            'preco': precos_ibov,
            'retorno': [0] + list(retornos_ibov[1:])
        })

        self.benchmarks_data['ibovespa'] = ibov_df

    def calcular_metricas_comparativas(self):
        """Calcula métricas de comparação com benchmarks"""
        # Preparar dados do fundo
        fundo_data = self.df_fundo[['data', 'cota', 'retorno_cota']].copy()
        fundo_data['data'] = pd.to_datetime(fundo_data['data'])

        resultados = {}

        for bench_name, bench_data in self.benchmarks_data.items():
            # Merge com dados do fundo
            merged = pd.merge(fundo_data, bench_data, on='data', how='inner', suffixes=('_fundo', '_bench'))

            if len(merged) < 10:  # Muito pouco dados
                continue

            # Calcular métricas
            ret_fundo = merged['retorno_cota'].dropna()
            ret_bench = merged['retorno'].dropna()

            if len(ret_fundo) != len(ret_bench) or len(ret_fundo) < 10:
                continue

            # Performance acumulada
            perf_fundo = (1 + ret_fundo).cumprod().iloc[-1] - 1
            perf_bench = (1 + ret_bench).cumprod().iloc[-1] - 1

            # Tracking Error
            excess_returns = ret_fundo - ret_bench
            tracking_error = excess_returns.std() * np.sqrt(252)

            # Information Ratio
            if tracking_error > 0:
                information_ratio = (excess_returns.mean() * 252) / tracking_error
            else:
                information_ratio = 0

            # Beta vs Benchmark
            if ret_bench.std() > 0:
                correlation = ret_fundo.corr(ret_bench)
                beta = (ret_fundo.cov(ret_bench)) / (ret_bench.var())
            else:
                correlation = 0
                beta = 0

            resultados[bench_name] = {
                'performance_fundo': perf_fundo * 100,
                'performance_benchmark': perf_bench * 100,
                'excess_return': (perf_fundo - perf_bench) * 100,
                'tracking_error': tracking_error * 100,
                'information_ratio': information_ratio,
                'correlation': correlation,
                'beta': beta,
                'dados_merged': merged
            }

        return resultados

    def stress_vs_benchmarks(self, cenario_stress, motor_stress):
        """Compara stress test do fundo vs benchmarks"""
        # Resultado do stress no fundo
        resultado_fundo = motor_stress.stress_test_cenario(cenario_stress)

        # Simular impacto nos benchmarks
        resultados_benchmarks = {}

        # CDI: pouco impactado por stress (safe haven)
        stress_cdi = max(-0.5, min(1.0, cenario_stress.get('selic', 0) * 0.01))  # 1% do stress da SELIC
        resultados_benchmarks['cdi'] = {
            'impacto_pct': stress_cdi,
            'nome': 'CDI'
        }

        # Ibovespa: usa diretamente o stress do ibovespa
        stress_ibov = cenario_stress.get('ibovespa', 0)
        resultados_benchmarks['ibovespa'] = {
            'impacto_pct': stress_ibov,
            'nome': 'Ibovespa'
        }

        # IPCA+: moderadamente impactado
        stress_ipca = (cenario_stress.get('selic', 0) * 0.3 + cenario_stress.get('dolar', 0) * 0.2) / 2
        resultados_benchmarks['ipca_plus'] = {
            'impacto_pct': stress_ipca,
            'nome': 'IPCA+'
        }

        # Adicionar resultado do fundo
        resultados_benchmarks['fundo'] = {
            'impacto_pct': resultado_fundo['impacto_total_pct'],
            'nome': 'Fundo'
        }

        return resultados_benchmarks

    def criar_grafico_benchmark_comparison(self, metricas_comparativas):
        """Cria gráfico de comparação com benchmarks"""
        if not metricas_comparativas:
            return None

        nomes = []
        performances = []
        cores = []

        for bench_name, metricas in metricas_comparativas.items():
            if bench_name == 'cdi':
                nome_display = '💰 CDI'
            elif bench_name == 'ibovespa':
                nome_display = '📈 Ibovespa'
            elif bench_name == 'ipca_plus':
                nome_display = '🎯 IPCA+'
            else:
                nome_display = bench_name.upper()

            nomes.append(nome_display)
            performances.append(metricas['performance_benchmark'])
            cores.append('#1f77b4')

        # Adicionar performance do fundo
        nomes.append('🏆 FUNDO')
        perf_fundo = list(metricas_comparativas.values())[0]['performance_fundo']  # Pegar do primeiro benchmark
        performances.append(perf_fundo)
        cores.append('#ff7f0e')

        fig = go.Figure(data=[
            go.Bar(
                x=nomes,
                y=performances,
                marker_color=cores,
                text=[f'{perf:+.1f}%' for perf in performances],
                textposition='auto',
                hovertemplate='%{x}<br>Performance: %{y:+.2f}%<extra></extra>'
            )
        ])

        fig.update_layout(
            title="🏆 Performance vs Benchmarks",
            xaxis_title="Assets",
            yaxis_title="Performance Acumulada (%)",
            template="plotly_white",
            height=400,
            showlegend=False
        )

        return fig

## 4.4. MOTOR INTEGRADO 2.0.

In [26]:
# 4.4. MOTOR INTEGRADO 2.0.

class MotorRiscoCompleto:
    """
    Motor integrado que combina todas as funcionalidades
    """

    def __init__(self, df_fundo, modelo_ibov, modelo_dolar, modelo_cupom, modelo_selic):
        # Motor básico de stress
        self.motor_stress = MotorStressTest(df_fundo, modelo_ibov, modelo_dolar, modelo_cupom, modelo_selic)

        # Componentes avançados
        self.drawdown_analyzer = DrawdownAnalyzer(df_fundo)
        self.stress_setorial = StressSetorial(self.motor_stress)
        self.benchmark_comparison = BenchmarkComparison(df_fundo)

        print("🚀 Motor de Risco Completo inicializado!")
        print("📊 Componentes disponíveis:")
        print("   • Análise de Drawdown")
        print("   • Stress Test Setorial")
        print("   • Comparação com Benchmarks")
        print("   • Dashboard Interativo 2.0")

    def analise_completa(self, cenario_stress):
        """Executa análise completa para um cenário"""
        resultados = {}

        # 1. Stress test básico
        resultados['stress_basico'] = self.motor_stress.stress_test_cenario(cenario_stress)

        # 2. Análise de drawdown
        resultados['drawdown_report'] = self.drawdown_analyzer.gerar_relatorio_drawdown()
        resultados['drawdown_stress'] = self.drawdown_analyzer.stress_test_drawdown(
            cenario_stress, self.motor_stress
        )

        # 3. Comparação com benchmarks
        resultados['benchmark_stress'] = self.benchmark_comparison.stress_vs_benchmarks(
            cenario_stress, self.motor_stress
        )

        return resultados

## 4.5. Interface Dashboard 2.0.

In [27]:
# 4.5. Interface Dashboard 2.0.

class Dashboard20:
    def __init__(self, motor_completo):
        self.motor = motor_completo

        # Interface mínima
        self.inputs = {
            'ibovespa': widgets.FloatText(value=0, description='Ibov %:'),
            'dolar': widgets.FloatText(value=0, description='Dólar %:'),
            'selic': widgets.FloatText(value=0, description='SELIC bps:'),
            'cupom': widgets.FloatText(value=0, description='Cupom bps:')
        }

        self.btn = widgets.Button(description='🔥 Calcular', button_style='primary')
        self.btn.on_click(self.calcular)

        self.output = widgets.Output()

        # Toggles opcionais
        self.toggle_drawdown = widgets.Checkbox(value=True, description='Drawdown')
        self.toggle_benchmarks = widgets.Checkbox(value=True, description='Benchmarks')
        self.toggle_setorial = widgets.Checkbox(value=True, description='Setorial')

    def mostrar(self):
        display(widgets.VBox([
            widgets.HBox(list(self.inputs.values())),
            widgets.HBox([self.toggle_drawdown, self.toggle_benchmarks, self.toggle_setorial]),
            self.btn,
            self.output
        ]))

    def calcular(self, b):
        with self.output:
            clear_output(wait=True)
            cenario = {k: v.value for k, v in self.inputs.items()}
            resultados = self.motor.analise_completa(cenario)

            # Resultado básico
            res = resultados['stress_basico']
            print(f"💰 R$ {res['cota_atual']:.4f} → R$ {res['cota_projetada']:.4f} ({res['impacto_total_pct']:+.2f}%)")

            # Gráficos (usa as correções já definidas)
            self._exibir_graficos_em_aba(resultados, cenario)

    # Placeholders para as correções
    def _exibir_graficos_em_aba(self, resultados, cenario): pass
    def _criar_grafico_drawdown_corrigido(self, ax): pass
    def _criar_grafico_benchmarks_corrigido(self, ax, resultados): pass
    def _criar_grafico_setorial_corrigido(self, ax): pass

In [28]:
# 5.5.1. Interface Dashboard 2.0.

# =============================================================================
# CORREÇÃO FINAL DOS GRÁFICOS - HIERARQUIA E LIMITES CORRETOS
# =============================================================================

# Substituir apenas a função _exibir_graficos_em_aba e as funções de criação dos gráficos:

def _exibir_graficos_em_aba(self, resultados, cenario):
    """Gráficos com hierarquia correta e limites adequados"""

    try:
        # TÍTULO PRINCIPAL - MAIS ACIMA NA HIERARQUIA
        display(widgets.HTML(f"""
        <div style="background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%);
                   padding: 20px; text-align: center; border-radius: 12px; margin-bottom: 30px;
                   font-weight: bold; color: white; font-size: 18px;
                   box-shadow: 0 4px 12px rgba(79, 172, 254, 0.3);">
            📈 ANÁLISE GRÁFICA DETALHADA
        </div>
        """))

        # Determinar quais gráficos serão exibidos
        graficos_ativos = []
        if self.toggle_drawdown.value:
            graficos_ativos.append('drawdown')
        if self.toggle_benchmarks.value:
            graficos_ativos.append('benchmarks')
        if self.toggle_setorial.value:
            graficos_ativos.append('setorial')

        if len(graficos_ativos) == 0:
            display(widgets.HTML("""
            <div style="text-align: center; padding: 60px; color: #666;
                       background: white; border-radius: 12px;
                       box-shadow: 0 4px 12px rgba(0,0,0,0.1);">
                <h3 style="color: #6c757d;">📊 Nenhum gráfico ativado</h3>
                <p style="color: #868e96;">Ative as opções de análise (Drawdown, Benchmarks, Setorial) para visualizar os gráficos.</p>
            </div>
            """))
            return

        # Container com background unificado para os gráficos
        display(widgets.HTML("""
        <div style="background: white; padding: 25px; border-radius: 15px;
                   box-shadow: 0 6px 20px rgba(0,0,0,0.12); margin-bottom: 20px;
                   border: 1px solid #e9ecef;">
        """))

        # Importar matplotlib
        import matplotlib.pyplot as plt
        plt.style.use('default')

        # TAMANHOS MENORES E MAIS COMPACTOS
        if len(graficos_ativos) == 1:
            fig, ax = plt.subplots(1, 1, figsize=(10, 3.5))
            axes = [ax]
        else:
            # GRÁFICOS MENORES: 5x3.5 cada um
            fig, axes = plt.subplots(1, 2, figsize=(11, 3.5))

        # Garantir que axes seja sempre uma lista
        if not isinstance(axes, (list, np.ndarray)):
            axes = [axes]

        # Configuração geral da figura
        fig.patch.set_facecolor('white')

        # REMOVER TÍTULO GERAL DA FIGURA (evita sobreposição)
        # fig.suptitle() - REMOVIDO PROPOSITALMENTE

        ax_idx = 0

        # 1. Gráfico de Drawdown
        if 'drawdown' in graficos_ativos:
            self._criar_grafico_drawdown_corrigido(axes[ax_idx])
            ax_idx += 1

        # 2. Gráfico de Benchmarks
        if 'benchmarks' in graficos_ativos:
            self._criar_grafico_benchmarks_corrigido(axes[ax_idx], resultados)
            ax_idx += 1

        # 3. Gráfico Setorial (se houver 3 gráficos, ajustar layout)
        if 'setorial' in graficos_ativos and ax_idx < len(axes):
            self._criar_grafico_setorial_corrigido(axes[ax_idx])
            ax_idx += 1

        # Esconder eixos não utilizados
        for i in range(ax_idx, len(axes)):
            axes[i].set_visible(False)

        # Layout otimizado - SEM SUPTITLE para evitar sobreposição
        plt.tight_layout(pad=2.0)  # Mais espaçamento
        plt.subplots_adjust(top=0.85, hspace=0.4, wspace=0.3)  # Mais espaço no topo
        plt.show()

        # Fechar container
        display(widgets.HTML("</div>"))

        print("✅ Gráficos corrigidos com hierarquia adequada!")

    except Exception as e:
        print(f"⚠️ Erro ao gerar gráficos: {e}")
        display(widgets.HTML(f"""
        <div style="text-align: center; padding: 40px; color: #d32f2f;
                   background: linear-gradient(135deg, #ffebee 0%, #ffcdd2 100%);
                   border-radius: 12px; border-left: 5px solid #d32f2f;
                   box-shadow: 0 4px 12px rgba(0,0,0,0.1);">
            <h4 style="color: #d32f2f; margin-bottom: 10px;">❌ Erro na Geração de Gráficos</h4>
            <p style="color: #c62828;">Erro: {str(e)}</p>
            <p style="color: #757575;"><em>As tabelas do Resumo Executivo foram carregadas com sucesso.</em></p>
        </div>
        """))

def _criar_grafico_drawdown_corrigido(self, ax):
    """Gráfico de drawdown com limites corretos e sem sobreposição"""
    try:
        dados = self.motor.drawdown_analyzer.df_fundo

        # Configuração visual refinada
        ax.fill_between(dados['data'], dados['drawdown_pct'], 0,
                       alpha=0.6, color='#e74c3c', label='Área de Drawdown')
        ax.plot(dados['data'], dados['drawdown_pct'],
               color='#c0392b', linewidth=2.5, label='Drawdown')

        # Título menor e mais limpo
        ax.set_title('Evolução do Drawdown - Análise Underwater',
                    fontsize=12, fontweight='bold', pad=15, color='#2c3e50')

        # AJUSTAR LIMITES AUTOMATICAMENTE COM MARGEM
        min_val = dados['drawdown_pct'].min()
        max_val = dados['drawdown_pct'].max()
        margem = abs(max_val - min_val) * 0.15  # 15% de margem

        ax.set_ylim(min_val - margem, max_val + margem)

        # Configurações visuais
        ax.axhline(y=0, color='#7f8c8d', linestyle='--', alpha=0.8, linewidth=1.5)
        ax.grid(True, alpha=0.4, linestyle='-', linewidth=0.5)
        ax.set_ylabel('Drawdown (%)', fontsize=10, fontweight='500', color='#34495e')

        # Anotação do máximo drawdown DENTRO DOS LIMITES
        max_dd_idx = dados['drawdown_pct'].idxmin()
        max_dd_value = dados['drawdown_pct'].iloc[max_dd_idx]
        max_dd_date = dados['data'].iloc[max_dd_idx]

        # Posicionamento da anotação que fica DENTRO do gráfico
        ax.annotate(f'Máx DD: {max_dd_value:.1f}%',
                   xy=(max_dd_date, max_dd_value),
                   xytext=(20, 25), textcoords='offset points',
                   bbox=dict(boxstyle='round,pad=0.4', facecolor='#f39c12', alpha=0.9, edgecolor='#e67e22'),
                   arrowprops=dict(arrowstyle='->', color='#e74c3c', lw=2),
                   fontsize=9, fontweight='bold', color='white')

        # Formatação dos eixos
        ax.tick_params(axis='x', rotation=30, labelsize=9, colors='#34495e')
        ax.tick_params(axis='y', labelsize=9, colors='#34495e')

        # Bordas e fundo
        ax.set_facecolor('#fafbfc')
        for spine in ax.spines.values():
            spine.set_color('#bdc3c7')
            spine.set_linewidth(1)

    except Exception as e:
        ax.text(0.5, 0.5, f'Erro no gráfico de drawdown:\n{str(e)}',
               transform=ax.transAxes, ha='center', va='center',
               bbox=dict(boxstyle='round,pad=0.5', facecolor='#e74c3c', alpha=0.8),
               fontsize=10, color='white', fontweight='bold')

def _criar_grafico_benchmarks_corrigido(self, ax, resultados):
    """Gráfico de benchmarks com limites corretos e valores dentro do gráfico"""
    try:
        bench_stress = resultados['benchmark_stress']
        nomes = [dados['nome'] for dados in bench_stress.values()]
        valores = [dados['impacto_pct'] for dados in bench_stress.values()]

        # Paleta de cores refinada
        cores = []
        for nome in nomes:
            if nome == 'Fundo':
                cores.append('#e67e22')  # Laranja vibrante
            elif 'CDI' in nome:
                cores.append('#27ae60')  # Verde
            elif 'Ibovespa' in nome:
                cores.append('#3498db')  # Azul
            else:
                cores.append('#9b59b6')  # Roxo

        # CALCULAR LIMITES COM MARGEM ADEQUADA
        min_val = min(valores)
        max_val = max(valores)
        margem = max(abs(max_val), abs(min_val)) * 0.25  # 25% de margem

        # Criar barras com visual refinado
        bars = ax.bar(nomes, valores, color=cores, alpha=0.85,
                     edgecolor='#2c3e50', linewidth=1.5)

        # Título menor e mais limpo
        ax.set_title('Performance Comparativa: Fundo vs Benchmarks',
                    fontsize=12, fontweight='bold', pad=15, color='#2c3e50')

        # DEFINIR LIMITES QUE ACOMODAM OS VALORES + TEXTO
        ax.set_ylim(min_val - margem, max_val + margem)

        # Configurações visuais
        ax.axhline(y=0, color='#7f8c8d', linestyle='--', alpha=0.8, linewidth=1.5)
        ax.grid(True, alpha=0.4, axis='y', linestyle='-', linewidth=0.5)
        ax.set_ylabel('Impacto (%)', fontsize=10, fontweight='500', color='#34495e')

        # Valores nas barras DENTRO DOS LIMITES
        for bar, valor in zip(bars, valores):
            height = bar.get_height()

            # Posicionamento inteligente do texto
            if valor >= 0:
                # Para valores positivos, colocar o texto ligeiramente acima da barra
                text_y = height + (margem * 0.1)
                va = 'bottom'
            else:
                # Para valores negativos, colocar o texto ligeiramente abaixo da barra
                text_y = height - (margem * 0.1)
                va = 'top'

            ax.text(bar.get_x() + bar.get_width()/2., text_y,
                   f'{valor:+.1f}%', ha='center', va=va,
                   fontweight='bold', fontsize=10, color='#2c3e50')

        # Destacar o fundo com borda especial
        for i, nome in enumerate(nomes):
            if nome == 'Fundo':
                bars[i].set_linewidth(3)
                bars[i].set_edgecolor('#d35400')
                bars[i].set_alpha(0.95)

        # Formatação dos eixos
        ax.tick_params(axis='x', rotation=15, labelsize=9, colors='#34495e')
        ax.tick_params(axis='y', labelsize=9, colors='#34495e')

        # Bordas e fundo
        ax.set_facecolor('#fafbfc')
        for spine in ax.spines.values():
            spine.set_color('#bdc3c7')
            spine.set_linewidth(1)

    except Exception as e:
        ax.text(0.5, 0.5, f'Erro no gráfico de benchmarks:\n{str(e)}',
               transform=ax.transAxes, ha='center', va='center',
               bbox=dict(boxstyle='round,pad=0.5', facecolor='#e74c3c', alpha=0.8),
               fontsize=10, color='white', fontweight='bold')

def _criar_grafico_setorial_corrigido(self, ax):
    """Gráfico setorial com limites corretos"""
    try:
        setores = ['financeiro_crise', 'commodities_crash', 'tech_crash', 'guerra_comercial']
        nomes = []
        valores = []

        for setor in setores:
            try:
                resultado = self.motor.stress_setorial.executar_stress_setorial(setor)
                nome_limpo = resultado['info_setor']['nome']
                # Limpar emojis para melhor legibilidade
                for emoji in ['🏦 ', '🛢️ ', '💻 ', '⚔️ ']:
                    nome_limpo = nome_limpo.replace(emoji, '')
                nomes.append(nome_limpo)
                valores.append(resultado['impacto_total_pct'])
            except:
                continue

        if nomes and valores:
            # CALCULAR LIMITES COM MARGEM
            min_val = min(valores)
            max_val = max(valores)
            margem = max(abs(max_val), abs(min_val)) * 0.25

            # Paleta de cores baseada na intensidade
            cores = []
            for v in valores:
                if v < -25:
                    cores.append('#8b0000')  # Vermelho escuro
                elif v < -10:
                    cores.append('#e74c3c')  # Vermelho
                elif v < 0:
                    cores.append('#f39c12')  # Laranja
                else:
                    cores.append('#27ae60')  # Verde

            bars = ax.bar(nomes, valores, color=cores, alpha=0.85,
                         edgecolor='#2c3e50', linewidth=1.5)

            # Título menor
            ax.set_title('Impacto por Setor Econômico',
                        fontsize=12, fontweight='bold', pad=15, color='#2c3e50')

            # DEFINIR LIMITES ADEQUADOS
            ax.set_ylim(min_val - margem, max_val + margem)

            # Configurações visuais
            ax.axhline(y=0, color='#7f8c8d', linestyle='--', alpha=0.8, linewidth=1.5)
            ax.grid(True, alpha=0.4, axis='y', linestyle='-', linewidth=0.5)
            ax.set_ylabel('Impacto (%)', fontsize=10, fontweight='500', color='#34495e')

            # Valores nas barras DENTRO DOS LIMITES
            for bar, valor in zip(bars, valores):
                height = bar.get_height()

                if valor >= 0:
                    text_y = height + (margem * 0.1)
                    va = 'bottom'
                else:
                    text_y = height - (margem * 0.1)
                    va = 'top'

                ax.text(bar.get_x() + bar.get_width()/2., text_y,
                       f'{valor:+.1f}%', ha='center', va=va,
                       fontweight='bold', fontsize=10, color='#2c3e50')

            # Formatação dos eixos
            ax.tick_params(axis='x', rotation=20, labelsize=9, colors='#34495e')
            ax.tick_params(axis='y', labelsize=9, colors='#34495e')

            # Bordas e fundo
            ax.set_facecolor('#fafbfc')
            for spine in ax.spines.values():
                spine.set_color('#bdc3c7')
                spine.set_linewidth(1)
        else:
            ax.text(0.5, 0.5, 'Dados setoriais\nnão disponíveis',
                   transform=ax.transAxes, ha='center', va='center',
                   fontsize=12, style='italic', color='#7f8c8d',
                   bbox=dict(boxstyle='round,pad=0.5', facecolor='#ecf0f1', alpha=0.8))
            ax.set_title('Impacto por Setor Econômico',
                        fontsize=12, fontweight='bold', pad=15, color='#2c3e50')

    except Exception as e:
        ax.text(0.5, 0.5, f'Erro no gráfico setorial:\n{str(e)}',
               transform=ax.transAxes, ha='center', va='center',
               bbox=dict(boxstyle='round,pad=0.5', facecolor='#e74c3c', alpha=0.8),
               fontsize=10, color='white', fontweight='bold')

# Aplicar as correções na classe Dashboard20
Dashboard20._exibir_graficos_em_aba = _exibir_graficos_em_aba
Dashboard20._criar_grafico_drawdown_corrigido = _criar_grafico_drawdown_corrigido
Dashboard20._criar_grafico_benchmarks_corrigido = _criar_grafico_benchmarks_corrigido
Dashboard20._criar_grafico_setorial_corrigido = _criar_grafico_setorial_corrigido

print("✅ CORREÇÕES FINAIS DOS GRÁFICOS APLICADAS!")
print()
print("🎯 Problemas Resolvidos:")
print("   • ✅ Hierarquia correta: Título principal acima dos gráficos")
print("   • ✅ Limites automáticos: Valores sempre dentro do gráfico")
print("   • ✅ Gráficos menores: 5x3.5 inches para mais espaço")
print("   • ✅ Títulos otimizados: Sem sobreposição")
print("   • ✅ Posicionamento inteligente: Texto sempre visível")
print()
print("📊 Especificações:")
print("   • Título principal: 18px, mais espaçamento")
print("   • Títulos dos gráficos: 12px, padding otimizado")
print("   • Limites automáticos: +25% margem para acomodar valores")
print("   • Layout: tight_layout com pad=2.0")
print()
print("🚀 Para aplicar: Execute esta célula e teste o dashboard!")

✅ CORREÇÕES FINAIS DOS GRÁFICOS APLICADAS!

🎯 Problemas Resolvidos:
   • ✅ Hierarquia correta: Título principal acima dos gráficos
   • ✅ Limites automáticos: Valores sempre dentro do gráfico
   • ✅ Gráficos menores: 5x3.5 inches para mais espaço
   • ✅ Títulos otimizados: Sem sobreposição
   • ✅ Posicionamento inteligente: Texto sempre visível

📊 Especificações:
   • Título principal: 18px, mais espaçamento
   • Títulos dos gráficos: 12px, padding otimizado
   • Limites automáticos: +25% margem para acomodar valores
   • Layout: tight_layout com pad=2.0

🚀 Para aplicar: Execute esta célula e teste o dashboard!


## 4.6. Funções de Conveniência e Testes Rápidos

In [29]:
# 4.6. Funções de Conveniência e Testes Rápidos

def criar_dashboard_completo(df_fundo, modelo_ibov, modelo_dolar, modelo_cupom, modelo_selic):
    """
    Função principal para criar o dashboard completo

    Usage:
    dashboard = criar_dashboard_completo(df, modelo_ibov, modelo_dolar, modelo_cupom, modelo_selic)
    dashboard.mostrar()
    """
    print("🚀 Inicializando Dashboard 2.0...")

    # Criar motor completo
    motor_completo = MotorRiscoCompleto(df_fundo, modelo_ibov, modelo_dolar, modelo_cupom, modelo_selic)

    # Criar interface
    dashboard = Dashboard20(motor_completo)

    print("✅ Dashboard 2.0 pronto para uso!")
    return dashboard

def teste_rapido_2_0(motor_completo, ibov=0, dolar=0, selic=0, cupom=0, mostrar_detalhes=True):
    """
    Teste rápido com funcionalidades 2.0

    Usage:
    teste_rapido_2_0(motor_completo, ibov=-15, dolar=20, selic=200, cupom=100)
    """
    cenario = {'ibovespa': ibov, 'dolar': dolar, 'selic': selic, 'cupom': cupom}

    print(f"⚡ Executando teste rápido: {cenario}")

    # Análise completa
    resultados = motor_completo.analise_completa(cenario)

    # Resultado básico
    res_basico = resultados['stress_basico']
    cor = "🔴" if res_basico['impacto_total_pct'] < 0 else "🟢"

    print(f"{cor} RESULTADO BÁSICO:")
    print(f"   Cota: R$ {res_basico['cota_atual']:.4f} → R$ {res_basico['cota_projetada']:.4f}")
    print(f"   Variação: {res_basico['impacto_total_pct']:+.2f}%")
    print(f"   P&L: R$ {res_basico['variacao_rs']:+.0f}")

    if mostrar_detalhes:
        # Drawdown
        dd_stress = resultados['drawdown_stress']
        print(f"\n📉 DRAWDOWN:")
        print(f"   Projetado: {dd_stress['drawdown_projetado_pct']:.2f}%")

        # Benchmarks
        bench_stress = resultados['benchmark_stress']
        print(f"\n🏆 vs BENCHMARKS:")
        for nome, dados in bench_stress.items():
            print(f"   {dados['nome']}: {dados['impacto_pct']:+.2f}%")

def analise_setorial_completa(motor_completo):
    """
    Executa análise completa de todos os setores
    """
    print("🔍 Executando análise setorial completa...")

    setores_crise = ['financeiro_crise', 'commodities_crash', 'tech_crash', 'guerra_comercial', 'hiperinflacao']
    setores_boom = ['financeiro_moderado', 'commodities_boom', 'tech_boom', 'paz_mundial']

    print("\n📉 CENÁRIOS DE CRISE:")
    for setor in setores_crise:
        try:
            resultado = motor_completo.stress_setorial.executar_stress_setorial(setor)
            nome = resultado['info_setor']['nome']
            impacto = resultado['impacto_total_pct']
            print(f"   {nome}: {impacto:+.2f}%")
        except:
            pass

    print("\n📈 CENÁRIOS POSITIVOS:")
    for setor in setores_boom:
        try:
            resultado = motor_completo.stress_setorial.executar_stress_setorial(setor)
            nome = resultado['info_setor']['nome']
            impacto = resultado['impacto_total_pct']
            print(f"   {nome}: {impacto:+.2f}%")
        except:
            pass

## 4.7. Relatórios Automáticos

In [30]:
# 4.7. Relatórios Automáticos

def gerar_relatorio_executivo(motor_completo, cenario, nome_arquivo=None):
    """
    Gera relatório executivo completo (versão texto para preview)
    """
    resultados = motor_completo.analise_completa(cenario)

    relatorio = f"""
    {'='*60}
    📊 RELATÓRIO EXECUTIVO DE RISCO
    {'='*60}

    🎯 CENÁRIO TESTADO:
    Ibovespa: {cenario['ibovespa']:+.1f}%
    Dólar: {cenario['dolar']:+.1f}%
    SELIC: {cenario['selic']:+.0f}bps
    Cupom: {cenario['cupom']:+.0f}bps

    💰 IMPACTO FINANCEIRO:
    Cota Atual: R$ {resultados['stress_basico']['cota_atual']:.4f}
    Cota Projetada: R$ {resultados['stress_basico']['cota_projetada']:.4f}
    Variação: {resultados['stress_basico']['impacto_total_pct']:+.2f}%
    P&L Projetado: R$ {resultados['stress_basico']['variacao_rs']:+,.0f}

    📉 ANÁLISE DE DRAWDOWN:
    Max Drawdown Histórico: {resultados['drawdown_report']['max_drawdown_pct']:.2f}%
    Drawdown Projetado: {resultados['drawdown_stress']['drawdown_projetado_pct']:.2f}%
    Tempo Underwater: {resultados['drawdown_report']['pct_tempo_underwater']:.1f}%

    🏆 COMPARAÇÃO BENCHMARKS:
    """

    for bench_name, bench_data in resultados['benchmark_stress'].items():
        relatorio += f"    {bench_data['nome']}: {bench_data['impacto_pct']:+.2f}%\n"

    relatorio += f"""
    ⚠️ RECOMENDAÇÕES:
    """

    impacto = resultados['stress_basico']['impacto_total_pct']
    if impacto > 5:
        relatorio += "    ✅ Risco baixo - Manter posição atual\n"
    elif impacto > -5:
        relatorio += "    ⚠️ Risco moderado - Monitorar de perto\n"
    elif impacto > -20:
        relatorio += "    🔴 Risco alto - Considerar hedge\n"
    else:
        relatorio += "    🚨 Risco crítico - Ação imediata necessária\n"

    relatorio += f"""
    {'='*60}
    Relatório gerado em: {datetime.now().strftime('%d/%m/%Y %H:%M')}
    """

    if nome_arquivo:
        with open(nome_arquivo, 'w', encoding='utf-8') as f:
            f.write(relatorio)
        print(f"📄 Relatório salvo em: {nome_arquivo}")

    return relatorio

## 4.8. Instruções de Uso

In [31]:
# 4.8. Instruções de Uso

"""DASHBOARD 2.0 - USO RÁPIDO:

# Setup:
dashboard = criar_dashboard_completo(df, modelo_ibov, modelo_dolar, modelo_cupom, modelo_selic)
dashboard.mostrar()

# Teste rápido:
teste_rapido_2_0(dashboard.motor, ibov=-15, dolar=20, selic=200, cupom=100)
"""

print("✅ Dashboard 2.0 pronto! Use: dashboard.mostrar()")

✅ Dashboard 2.0 pronto! Use: dashboard.mostrar()


# 5.0. DASHBOARD OUTPUT

## 5.1. DASHBOARD CORRIGIDO - COM ABAS

In [32]:
# 5.1. DASHBOARD CORRIGIDO - VERSÃO FINAL
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import numpy as np

class DashboardComAbas:
    def __init__(self, motor_completo):
        self.motor = motor_completo
        self.criar_interface_completa()

    def criar_interface_completa(self):
        """Interface completa com abas e cenários históricos"""

        # === TÍTULO PRINCIPAL ===
        titulo = widgets.HTML(f"""
        <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                   padding: 15px; text-align: center; border-radius: 10px; margin-bottom: 20px;
                   color: white; font-size: 18px; font-weight: bold;
                   box-shadow: 0 4px 15px rgba(102, 126, 234, 0.3);">
            🎯 DASHBOARD DE RISCO 2.0 | Cota Atual: R$ {self.motor.motor_stress.cota_atual:.4f}
        </div>
        """)

        # === INPUTS DOS FATORES ===
        self.inputs = {
            'ibovespa': widgets.FloatText(value=0, description='Ibovespa (%):',
                                        style={'description_width': '100px'}, layout={'width': '180px'}),
            'dolar': widgets.FloatText(value=0, description='Dólar (%):',
                                     style={'description_width': '100px'}, layout={'width': '180px'}),
            'selic': widgets.FloatText(value=0, description='SELIC (bps):',
                                     style={'description_width': '100px'}, layout={'width': '180px'}),
            'cupom': widgets.FloatText(value=0, description='Cupom (bps):',
                                     style={'description_width': '100px'}, layout={'width': '180px'})
        }

        # === CENÁRIOS HISTÓRICOS ===
        self.dropdown_cenarios = widgets.Dropdown(
            options=[
                ('🔹 Selecione um cenário...', 'neutro'),
                ('--- CENÁRIOS NEGATIVOS ---', 'separator1'),
                ('🔴 Impeachment Dilma (2016)', 'impeachment'),
                ('💥 Joesley Day (2017)', 'joesley'),
                ('🦠 COVID-19 (Mar 2020)', 'covid'),
                ('📉 Crise Subprime (2008)', 'subprime'),
                ('⚡ Guerra Rússia-Ucrânia (2022)', 'guerra'),
                ('--- CENÁRIOS POSITIVOS ---', 'separator2'),
                ('🏆 Copa do Mundo 2002 + Lula', 'copa2002'),
                ('⭐ Boom Commodities (2003-2007)', 'boom_commodities'),
                ('💎 Descoberta Pré-Sal (2007)', 'presal'),
                ('🎯 Plano Real Consolidado (1999)', 'plano_real'),
                ('📈 Rally Pós-Eleição Lula (2002)', 'pos_lula'),
                ('🌟 Grau Investimento (2008)', 'grau_investimento')
            ],
            value='neutro',
            description='Cenários Históricos:',
            style={'description_width': '130px'},
            layout={'width': '400px'}
        )

        # === STRESS SETORIAL ===
        self.dropdown_setorial = widgets.Dropdown(
            options=[
                ('🔹 Selecione setor...', 'neutro'),
                ('🏦 Crise Bancária', 'financeiro_crise'),
                ('🏦 Stress Financeiro Moderado', 'financeiro_moderado'),
                ('🛢️ Crash das Commodities', 'commodities_crash'),
                ('🛢️ Boom das Commodities', 'commodities_boom'),
                ('💻 Crash Tecnológico', 'tech_crash'),
                ('💻 Boom Tecnológico', 'tech_boom'),
                ('⚔️ Guerra Comercial', 'guerra_comercial'),
                ('🕊️ Paz Mundial', 'paz_mundial'),
                ('📈 Hiperinflação', 'hiperinflacao'),
                ('📉 Deflação', 'deflacao')
            ],
            value='neutro',
            description='Stress Setorial:',
            style={'description_width': '100px'},
            layout={'width': '300px'}
        )

        # === BOTÕES ===
        self.btn_calcular = widgets.Button(description='⚡ CALCULAR', button_style='primary',
                                         layout={'width': '120px'})
        self.btn_limpar = widgets.Button(description='🔄 Limpar', button_style='warning',
                                       layout={'width': '100px'})

        # === CHECKBOXES PARA GRÁFICOS ===
        self.check_drawdown = widgets.Checkbox(value=True, description='Incluir Análise Drawdown')
        self.check_benchmarks = widgets.Checkbox(value=True, description='Comparar com Benchmarks')
        self.check_setorial = widgets.Checkbox(value=True, description='Análise Setorial Comparativa')

        # === ABAS ===
        self.aba_resumo = widgets.Output()
        self.aba_graficos = widgets.Output()

        self.tabs = widgets.Tab()
        self.tabs.children = [self.aba_resumo, self.aba_graficos]
        self.tabs.set_title(0, '📊 Resumo Executivo')
        self.tabs.set_title(1, '📈 Gráficos')

        # === EVENTOS ===
        self.btn_calcular.on_click(self.calcular_completo)
        self.btn_limpar.on_click(self.limpar)
        self.dropdown_cenarios.observe(self.carregar_cenario_historico, names='value')
        self.dropdown_setorial.observe(self.carregar_cenario_setorial, names='value')

        # === LAYOUT FINAL ===
        linha_inputs = widgets.HBox(list(self.inputs.values()))
        linha_cenarios = widgets.HBox([self.dropdown_cenarios, self.dropdown_setorial])
        linha_botoes = widgets.HBox([self.btn_calcular, self.btn_limpar])
        linha_checks = widgets.HBox([self.check_drawdown, self.check_benchmarks, self.check_setorial])

        self.interface = widgets.VBox([
            titulo,
            linha_inputs,
            linha_cenarios,
            linha_botoes,
            linha_checks,
            self.tabs
        ])

    def carregar_cenario_historico(self, change):
        """Carrega cenários históricos brasileiros"""
        cenarios_historicos = {
            # CENÁRIOS NEGATIVOS
            'impeachment': {'ibovespa': -12.0, 'dolar': 18.0, 'selic': 200, 'cupom': 150},
            'joesley': {'ibovespa': -8.5, 'dolar': 12.0, 'selic': 0, 'cupom': 80},
            'covid': {'ibovespa': -35.0, 'dolar': 25.0, 'selic': -200, 'cupom': 300},
            'subprime': {'ibovespa': -28.0, 'dolar': 22.0, 'selic': 300, 'cupom': 250},
            'guerra': {'ibovespa': -15.0, 'dolar': 8.0, 'selic': 150, 'cupom': 100},

            # CENÁRIOS POSITIVOS
            'copa2002': {'ibovespa': 25.0, 'dolar': -15.0, 'selic': -150, 'cupom': -100},
            'boom_commodities': {'ibovespa': 35.0, 'dolar': -20.0, 'selic': -200, 'cupom': -150},
            'presal': {'ibovespa': 18.0, 'dolar': -12.0, 'selic': -100, 'cupom': -80},
            'plano_real': {'ibovespa': 22.0, 'dolar': -18.0, 'selic': -300, 'cupom': -200},
            'pos_lula': {'ibovespa': 28.0, 'dolar': -16.0, 'selic': -180, 'cupom': -120},
            'grau_investimento': {'ibovespa': 20.0, 'dolar': -14.0, 'selic': -120, 'cupom': -90}
        }

        if change['new'].startswith('separator'):
            return

        if change['new'] in cenarios_historicos:
            cenario = cenarios_historicos[change['new']]
            self.inputs['ibovespa'].value = cenario['ibovespa']
            self.inputs['dolar'].value = cenario['dolar']
            self.inputs['selic'].value = cenario['selic']
            self.inputs['cupom'].value = cenario['cupom']
            # Reset setorial quando escolher histórico
            self.dropdown_setorial.value = 'neutro'
        elif change['new'] == 'neutro':
            self.limpar_valores()

    def carregar_cenario_setorial(self, change):
        """Carrega cenários setoriais"""
        if change['new'] == 'neutro' or change['new'].startswith('separator'):
            return

        try:
            resultado = self.motor.stress_setorial.executar_stress_setorial(change['new'])
            cenario = resultado['info_setor']['cenario']

            self.inputs['ibovespa'].value = cenario['ibovespa']
            self.inputs['dolar'].value = cenario['dolar']
            self.inputs['selic'].value = cenario['selic']
            self.inputs['cupom'].value = cenario['cupom']
            # Reset histórico quando escolher setorial
            self.dropdown_cenarios.value = 'neutro'
        except:
            pass

    def calcular_completo(self, b):
        """Calcula e exibe nas duas abas"""
        cenario = {k: v.value for k, v in self.inputs.items()}

        # Executar análise completa
        resultados = self.motor.analise_completa(cenario)

        # Atualizar aba de resumo
        self.atualizar_aba_resumo(resultados, cenario)

        # Atualizar aba de gráficos
        self.atualizar_aba_graficos(resultados, cenario)

    def atualizar_aba_resumo(self, resultados, cenario):
        """Atualiza a aba de resumo executivo - LAYOUT CORRIGIDO"""
        with self.aba_resumo:
            clear_output(wait=True)

            res = resultados['stress_basico']

            # Semáforo de risco
            variacao_pct = res['impacto_total_pct']
            if variacao_pct > 5:
                semaforo = "🟢"
                nivel_risco = "BAIXO"
                cor_semaforo = "#27ae60"
                recomendacao = "✅ MANTER POSIÇÃO"
            elif variacao_pct > -5:
                semaforo = "🟡"
                nivel_risco = "MODERADO"
                cor_semaforo = "#f39c12"
                recomendacao = "⚠️ MONITORAR"
            elif variacao_pct > -20:
                semaforo = "🔴"
                nivel_risco = "ALTO"
                cor_semaforo = "#e74c3c"
                recomendacao = "🛡️ CONSIDERAR HEDGE"
            else:
                semaforo = "⚫"
                nivel_risco = "CRÍTICO"
                cor_semaforo = "#2c3e50"
                recomendacao = "🚨 AÇÃO IMEDIATA"

            cor_fundo = "#ffebee" if res['impacto_total_pct'] < 0 else "#e8f5e8"
            cor_texto = "#c62828" if res['impacto_total_pct'] < 0 else "#2e7d32"

            # Cenário testado
            cenario_str = " | ".join([f"{k.upper()}: {v:+}" + ("%" if k in ['ibovespa', 'dolar'] else "bps")
                                    for k, v in cenario.items() if v != 0]) or "NEUTRO"

            # ===== INÍCIO DO HTML CORRIGIDO =====
            html_resumo = f"""
            <div style="font-family: Arial, sans-serif; max-width: 1000px; margin: 0 auto; background: #f8f9fa; padding: 25px; border-radius: 15px; box-shadow: 0 6px 12px rgba(0,0,0,0.1);">

                <!-- Dashboard Executivo - PRIMEIRO QUADRO -->
                <div style="display: flex; gap: 20px; margin-bottom: 25px;">

                    <!-- Semáforo de Risco -->
                    <div style="background: white; border-radius: 12px; padding: 25px; box-shadow: 0 4px 8px rgba(0,0,0,0.1); text-align: center; border-left: 6px solid {cor_semaforo}; min-width: 220px;">
                        <div style="font-size: 56px; margin-bottom: 15px;">{semaforo}</div>
                        <h3 style="color: {cor_semaforo}; margin: 8px 0; font-size: 20px;">RISCO {nivel_risco}</h3>
                        <p style="color: {cor_semaforo}; font-weight: bold; margin: 8px 0; font-size: 15px;">{recomendacao}</p>
                    </div>

                    <!-- Resumo Financeiro -->
                    <div style="background: white; border-radius: 12px; padding: 25px; box-shadow: 0 4px 8px rgba(0,0,0,0.1); flex: 1;">
                        <h3 style="color: #333; margin-top: 0; font-size: 18px; margin-bottom: 20px;">📊 RESUMO EXECUTIVO</h3>
                        <div style="display: flex; justify-content: space-between; align-items: center; gap: 20px;">
                            <div style="text-align: center;">
                                <p style="margin: 5px 0; color: #666; font-size: 14px;">Impacto 21 dias:</p>
                                <h2 style="margin: 5px 0; color: {cor_texto}; font-size: 28px;">{res['impacto_total_pct']:+.2f}%</h2>
                            </div>
                            <div style="text-align: center;">
                                <p style="margin: 5px 0; color: #666; font-size: 14px;">P&L Projetado:</p>
                                <h2 style="margin: 5px 0; color: {cor_texto}; font-size: 24px;">R$ {res['variacao_rs']:+.0f}</h2>
                            </div>
                            <div style="text-align: center;">
                                <p style="margin: 5px 0; color: #666; font-size: 14px;">VaR 95%:</p>
                                <h2 style="margin: 5px 0; color: #e67e22; font-size: 24px;">R$ {res['var_95_rs']:.0f}</h2>
                            </div>
                        </div>
                    </div>
                </div>

                <!-- Cenário testado -->
                <div style="background: white; padding: 18px; text-align: center; border-radius: 12px; margin-bottom: 25px; font-weight: bold; color: #333; font-size: 17px; border-left: 5px solid #007bff; box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
                    🎯 CENÁRIO TESTADO: {cenario_str}
                </div>

                <!-- SEGUNDO QUADRO - Métricas Principais e Breakdown lado a lado -->
                <div style="display: flex; gap: 20px; margin-bottom: 25px;">

                    <!-- Tabela Principal -->
                    <div style="flex: 1;">
                        <table style="width: 100%; border-collapse: collapse; background: white; box-shadow: 0 4px 8px rgba(0,0,0,0.1); border-radius: 12px; overflow: hidden;">
                            <thead>
                                <tr style="background: {cor_fundo};">
                                    <th style="padding: 18px; text-align: left; color: #333; font-size: 16px; font-weight: bold;">📊 MÉTRICAS PRINCIPAIS</th>
                                    <th style="padding: 18px; text-align: right; color: #333; font-size: 16px; font-weight: bold;">VALOR</th>
                                </tr>
                            </thead>
                            <tbody>
                                <tr style="background: #fafafa;">
                                    <td style="padding: 15px 18px; color: #333; font-size: 15px;">💰 Cota Atual</td>
                                    <td style="padding: 15px 18px; text-align: right; font-weight: bold; color: #333; font-size: 16px;">R$ {res['cota_atual']:.4f}</td>
                                </tr>
                                <tr>
                                    <td style="padding: 15px 18px; color: #333; font-size: 15px;">💰 Cota Projetada</td>
                                    <td style="padding: 15px 18px; text-align: right; font-weight: bold; color: {cor_texto}; font-size: 18px;">R$ {res['cota_projetada']:.4f}</td>
                                </tr>
                                <tr style="background: #fafafa;">
                                    <td style="padding: 15px 18px; color: #333; font-size: 15px;">📈 Variação (%)</td>
                                    <td style="padding: 15px 18px; text-align: right; font-weight: bold; color: {cor_texto}; font-size: 18px;">{res['impacto_total_pct']:+.2f}%</td>
                                </tr>
                                <tr>
                                    <td style="padding: 15px 18px; color: #333; font-size: 15px;">💵 P&L Total</td>
                                    <td style="padding: 15px 18px; text-align: right; font-weight: bold; color: {cor_texto}; font-size: 16px;">R$ {res['variacao_rs']:+.0f}</td>
                                </tr>
                                <tr style="background: #fafafa;">
                                    <td style="padding: 15px 18px; color: #333; font-size: 15px;">⚠️ VaR 95%</td>
                                    <td style="padding: 15px 18px; text-align: right; font-weight: bold; color: #ff6f00; font-size: 16px;">R$ {res['var_95_rs']:.0f}</td>
                                </tr>
                            </tbody>
                        </table>
                    </div>

                    <!-- Breakdown dos Fatores -->
                    <div style="flex: 1;">
                        <table style="width: 100%; border-collapse: collapse; background: white; box-shadow: 0 4px 8px rgba(0,0,0,0.1); border-radius: 12px; overflow: hidden;">
                            <thead>
                                <tr style="background: #e3f2fd;">
                                    <th style="padding: 18px 15px; text-align: left; color: #333; font-size: 16px; font-weight: bold;">🔍 BREAKDOWN POR FATOR</th>
                                    <th style="padding: 18px 10px; text-align: center; color: #333; font-size: 16px; font-weight: bold;">CENÁRIO</th>
                                    <th style="padding: 18px 15px; text-align: right; color: #333; font-size: 16px; font-weight: bold;">IMPACTO</th>
                                </tr>
                            </thead>
                            <tbody>
            """

            # Breakdown dos fatores
            if res['impactos_individuais']:
                cenario_display = {
                    'ibovespa': f"{cenario['ibovespa']:+.1f}%",
                    'dolar': f"{cenario['dolar']:+.1f}%",
                    'selic': f"{cenario['selic']:+.0f}bps",
                    'cupom': f"{cenario['cupom']:+.0f}bps"
                }

                for i, (fator, impacto) in enumerate(res['impactos_individuais'].items()):
                    cor_linha = "#fafafa" if i % 2 == 0 else "white"
                    cor_impacto = "#c62828" if impacto < 0 else "#27ae60"
                    emoji = "📉" if impacto < 0 else "📈"

                    html_resumo += f"""
                        <tr style="background: {cor_linha};">
                            <td style="padding: 15px; color: #333; font-size: 15px;">{emoji} {fator.capitalize()}</td>
                            <td style="padding: 15px; text-align: center; color: #666; font-size: 15px; font-weight: 500;">{cenario_display.get(fator, '-')}</td>
                            <td style="padding: 15px; text-align: right; font-weight: bold; color: {cor_impacto}; font-size: 16px;">{impacto:+.2%}</td>
                        </tr>
                    """

            html_resumo += """
                            </tbody>
                        </table>
                    </div>
                </div>
            """

            # ===== TERCEIRO QUADRO - DRAWDOWN (SEPARADO) =====
            if 'drawdown_report' in resultados:
                dd_report = resultados['drawdown_report']
                dd_stress = resultados['drawdown_stress']

                html_resumo += f"""
                <!-- TERCEIRO QUADRO - Seção Drawdown SEPARADA -->
                <div style="background: white; border-radius: 12px; padding: 25px; margin-bottom: 20px; box-shadow: 0 4px 8px rgba(0,0,0,0.1); border-left: 5px solid #e74c3c;">
                    <h3 style="color: #333; margin-top: 0; font-size: 18px; margin-bottom: 20px; display: flex; align-items: center;">
                        <span style="margin-right: 10px;">📉</span> DRAWDOWN
                    </h3>
                    <div style="display: flex; justify-content: space-around; text-align: center; gap: 30px;">
                        <div style="flex: 1; padding: 20px; background: #fef5f5; border-radius: 10px; border: 2px solid #ffebee;">
                            <p style="margin: 5px 0; color: #666; font-size: 14px; font-weight: 500;">Máx Histórico:</p>
                            <h3 style="margin: 10px 0; color: #e74c3c; font-size: 24px; font-weight: bold;">{dd_report['max_drawdown_pct']:.2f}%</h3>
                            <p style="margin: 5px 0; color: #999; font-size: 12px;">Pior cenário observado</p>
                        </div>
                        <div style="flex: 1; padding: 20px; background: #fff3e0; border-radius: 10px; border: 2px solid #ffe0b2;">
                            <p style="margin: 5px 0; color: #666; font-size: 14px; font-weight: 500;">Projetado:</p>
                            <h3 style="margin: 10px 0; color: #f39c12; font-size: 24px; font-weight: bold;">{dd_stress['drawdown_projetado_pct']:.2f}%</h3>
                            <p style="margin: 5px 0; color: #999; font-size: 12px;">Para este cenário</p>
                        </div>
                    </div>
                </div>
                """

            # ===== QUARTO QUADRO - BENCHMARKS (SEPARADO) =====
            if 'benchmark_stress' in resultados:
                html_resumo += f"""
                <!-- QUARTO QUADRO - Benchmarks SEPARADO -->
                <div style="background: white; border-radius: 12px; padding: 25px; margin-bottom: 20px; box-shadow: 0 4px 8px rgba(0,0,0,0.1); border-left: 5px solid #3498db;">
                    <h3 style="color: #333; margin-top: 0; font-size: 18px; margin-bottom: 20px; display: flex; align-items: center;">
                        <span style="margin-right: 10px;">🏆</span> vs BENCHMARKS
                    </h3>
                    <div style="display: flex; justify-content: space-around; gap: 15px; flex-wrap: wrap;">
                """

                for bench_name, bench_data in resultados['benchmark_stress'].items():
                    cor_bench = "#27ae60" if bench_data['impacto_pct'] >= 0 else "#e74c3c"
                    cor_fundo_bench = "#f0f8f0" if bench_data['impacto_pct'] >= 0 else "#fef5f5"
                    border_color = "#c8e6c9" if bench_data['impacto_pct'] >= 0 else "#ffcdd2"

                    # Emoji baseado no tipo de benchmark
                    if 'CDI' in bench_data['nome']:
                        emoji = "💰"
                    elif 'Ibovespa' in bench_data['nome']:
                        emoji = "📈"
                    elif 'IPCA' in bench_data['nome']:
                        emoji = "🎯"
                    else:
                        emoji = "🏆"

                    html_resumo += f"""
                    <div style="flex: 1; min-width: 140px; padding: 18px; background: {cor_fundo_bench}; border-radius: 10px; border: 2px solid {border_color}; text-align: center; margin-bottom: 10px;">
                        <div style="font-size: 24px; margin-bottom: 8px;">{emoji}</div>
                        <p style="margin: 5px 0; color: #666; font-size: 13px; font-weight: 500;">{bench_data['nome']}</p>
                        <h4 style="margin: 8px 0; color: {cor_bench}; font-size: 18px; font-weight: bold;">{bench_data['impacto_pct']:+.1f}%</h4>
                    </div>
                    """

                html_resumo += """
                    </div>
                </div>
                """

            # Fechar container principal
            html_resumo += "</div>"

            display(widgets.HTML(html_resumo))

    def atualizar_aba_graficos(self, resultados, cenario):
        """Atualiza a aba de gráficos"""
        with self.aba_graficos:
            clear_output(wait=True)

            try:
                # Título da seção gráficos
                display(widgets.HTML(f"""
                <div style="background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%);
                           padding: 20px; text-align: center; border-radius: 12px; margin-bottom: 30px;
                           font-weight: bold; color: white; font-size: 18px;
                           box-shadow: 0 4px 12px rgba(79, 172, 254, 0.3);">
                    📈 ANÁLISE GRÁFICA DETALHADA
                </div>
                """))

                # Determinar quais gráficos exibir
                graficos_ativos = []
                if self.check_drawdown.value:
                    graficos_ativos.append('drawdown')
                if self.check_benchmarks.value:
                    graficos_ativos.append('benchmarks')
                if self.check_setorial.value:
                    graficos_ativos.append('setorial')

                if not graficos_ativos:
                    display(widgets.HTML("""
                    <div style="text-align: center; padding: 60px; color: #666;
                               background: white; border-radius: 12px;
                               box-shadow: 0 4px 12px rgba(0,0,0,0.1);">
                        <h3 style="color: #6c757d;">📊 Nenhum gráfico ativado</h3>
                        <p style="color: #868e96;">Ative as opções de análise para visualizar os gráficos.</p>
                    </div>
                    """))
                    return

                # Importar matplotlib
                import matplotlib.pyplot as plt
                plt.style.use('default')

                # Layout dos gráficos
                if len(graficos_ativos) == 1:
                    fig, ax = plt.subplots(1, 1, figsize=(12, 4))
                    axes = [ax]
                elif len(graficos_ativos) == 2:
                    fig, axes = plt.subplots(1, 2, figsize=(15, 4))
                else:
                    fig, axes = plt.subplots(2, 2, figsize=(15, 8))
                    axes = axes.flatten()

                if not isinstance(axes, (list, np.ndarray)):
                    axes = [axes]

                fig.patch.set_facecolor('white')
                ax_idx = 0

                # Gráfico de Drawdown
                if 'drawdown' in graficos_ativos:
                    self._criar_grafico_drawdown(axes[ax_idx])
                    ax_idx += 1

                # Gráfico de Benchmarks
                if 'benchmarks' in graficos_ativos:
                    self._criar_grafico_benchmarks(axes[ax_idx], resultados)
                    ax_idx += 1

                # Gráfico Setorial
                if 'setorial' in graficos_ativos:
                    self._criar_grafico_setorial(axes[ax_idx])
                    ax_idx += 1

                # Esconder eixos não utilizados
                for i in range(ax_idx, len(axes)):
                    axes[i].set_visible(False)

                plt.tight_layout(pad=3.0)
                plt.show()

            except Exception as e:
                display(widgets.HTML(f"""
                <div style="text-align: center; padding: 40px; color: #d32f2f;
                           background: linear-gradient(135deg, #ffebee 0%, #ffcdd2 100%);
                           border-radius: 12px; border-left: 5px solid #d32f2f;
                           box-shadow: 0 4px 12px rgba(0,0,0,0.1);">
                    <h4 style="color: #d32f2f; margin-bottom: 10px;">❌ Erro na Geração de Gráficos</h4>
                    <p style="color: #c62828;">Erro: {str(e)}</p>
                </div>
                """))

    def _criar_grafico_drawdown(self, ax):
        """Cria gráfico de drawdown"""
        try:
            dados = self.motor.drawdown_analyzer.df_fundo

            ax.fill_between(dados['data'], dados['drawdown_pct'], 0,
                           alpha=0.6, color='#e74c3c', label='Área de Drawdown')
            ax.plot(dados['data'], dados['drawdown_pct'],
                   color='#c0392b', linewidth=2.5, label='Drawdown')

            ax.set_title('Evolução do Drawdown - Análise Underwater',
                        fontsize=14, fontweight='bold', pad=15, color='#2c3e50')

            min_val = dados['drawdown_pct'].min()
            max_val = dados['drawdown_pct'].max()
            margem = abs(max_val - min_val) * 0.15
            ax.set_ylim(min_val - margem, max_val + margem)

            ax.axhline(y=0, color='#7f8c8d', linestyle='--', alpha=0.8, linewidth=1.5)
            ax.grid(True, alpha=0.4, linestyle='-', linewidth=0.5)
            ax.set_ylabel('Drawdown (%)', fontsize=11, fontweight='500', color='#34495e')

            # Anotação do máximo drawdown
            max_dd_idx = dados['drawdown_pct'].idxmin()
            max_dd_value = dados['drawdown_pct'].iloc[max_dd_idx]
            max_dd_date = dados['data'].iloc[max_dd_idx]

            ax.annotate(f'Máx DD: {max_dd_value:.1f}%',
                       xy=(max_dd_date, max_dd_value),
                       xytext=(20, 25), textcoords='offset points',
                       bbox=dict(boxstyle='round,pad=0.4', facecolor='#f39c12', alpha=0.9, edgecolor='#e67e22'),
                       arrowprops=dict(arrowstyle='->', color='#e74c3c', lw=2),
                       fontsize=10, fontweight='bold', color='white')

            ax.tick_params(axis='x', rotation=30, labelsize=10, colors='#34495e')
            ax.tick_params(axis='y', labelsize=10, colors='#34495e')

            ax.set_facecolor('#fafbfc')
            for spine in ax.spines.values():
                spine.set_color('#bdc3c7')
                spine.set_linewidth(1)

        except Exception as e:
            ax.text(0.5, 0.5, f'Erro no gráfico de drawdown:\n{str(e)}',
                   transform=ax.transAxes, ha='center', va='center',
                   bbox=dict(boxstyle='round,pad=0.5', facecolor='#e74c3c', alpha=0.8),
                   fontsize=10, color='white', fontweight='bold')

    def _criar_grafico_benchmarks(self, ax, resultados):
        """Cria gráfico de benchmarks"""
        try:
            bench_stress = resultados['benchmark_stress']
            nomes = [dados['nome'] for dados in bench_stress.values()]
            valores = [dados['impacto_pct'] for dados in bench_stress.values()]

            cores = []
            for nome in nomes:
                if nome == 'Fundo':
                    cores.append('#e67e22')
                elif 'CDI' in nome:
                    cores.append('#27ae60')
                elif 'Ibovespa' in nome:
                    cores.append('#3498db')
                else:
                    cores.append('#9b59b6')

            min_val = min(valores)
            max_val = max(valores)
            margem = max(abs(max_val), abs(min_val)) * 0.25

            bars = ax.bar(nomes, valores, color=cores, alpha=0.85,
                         edgecolor='#2c3e50', linewidth=1.5)

            ax.set_title('Performance Comparativa: Fundo vs Benchmarks',
                        fontsize=14, fontweight='bold', pad=15, color='#2c3e50')

            ax.set_ylim(min_val - margem, max_val + margem)
            ax.axhline(y=0, color='#7f8c8d', linestyle='--', alpha=0.8, linewidth=1.5)
            ax.grid(True, alpha=0.4, axis='y', linestyle='-', linewidth=0.5)
            ax.set_ylabel('Impacto (%)', fontsize=11, fontweight='500', color='#34495e')

            # Valores nas barras
            for bar, valor in zip(bars, valores):
                height = bar.get_height()
                if valor >= 0:
                    text_y = height + (margem * 0.1)
                    va = 'bottom'
                else:
                    text_y = height - (margem * 0.1)
                    va = 'top'

                ax.text(bar.get_x() + bar.get_width()/2., text_y,
                       f'{valor:+.1f}%', ha='center', va=va,
                       fontweight='bold', fontsize=10, color='#2c3e50')

            # Destacar o fundo
            for i, nome in enumerate(nomes):
                if nome == 'Fundo':
                    bars[i].set_linewidth(3)
                    bars[i].set_edgecolor('#d35400')
                    bars[i].set_alpha(0.95)

            ax.tick_params(axis='x', rotation=15, labelsize=10, colors='#34495e')
            ax.tick_params(axis='y', labelsize=10, colors='#34495e')

            ax.set_facecolor('#fafbfc')
            for spine in ax.spines.values():
                spine.set_color('#bdc3c7')
                spine.set_linewidth(1)

        except Exception as e:
            ax.text(0.5, 0.5, f'Erro no gráfico de benchmarks:\n{str(e)}',
                   transform=ax.transAxes, ha='center', va='center',
                   bbox=dict(boxstyle='round,pad=0.5', facecolor='#e74c3c', alpha=0.8),
                   fontsize=10, color='white', fontweight='bold')

    def _criar_grafico_setorial(self, ax):
        """Cria gráfico setorial"""
        try:
            setores = ['financeiro_crise', 'commodities_crash', 'tech_crash', 'guerra_comercial']
            nomes = []
            valores = []

            for setor in setores:
                try:
                    resultado = self.motor.stress_setorial.executar_stress_setorial(setor)
                    nome_limpo = resultado['info_setor']['nome']
                    for emoji in ['🏦 ', '🛢️ ', '💻 ', '⚔️ ']:
                        nome_limpo = nome_limpo.replace(emoji, '')
                    nomes.append(nome_limpo)
                    valores.append(resultado['impacto_total_pct'])
                except:
                    continue

            if nomes and valores:
                min_val = min(valores)
                max_val = max(valores)
                margem = max(abs(max_val), abs(min_val)) * 0.25

                cores = []
                for v in valores:
                    if v < -25:
                        cores.append('#8b0000')
                    elif v < -10:
                        cores.append('#e74c3c')
                    elif v < 0:
                        cores.append('#f39c12')
                    else:
                        cores.append('#27ae60')

                bars = ax.bar(nomes, valores, color=cores, alpha=0.85,
                             edgecolor='#2c3e50', linewidth=1.5)

                ax.set_title('Impacto por Setor Econômico',
                            fontsize=14, fontweight='bold', pad=15, color='#2c3e50')

                ax.set_ylim(min_val - margem, max_val + margem)
                ax.axhline(y=0, color='#7f8c8d', linestyle='--', alpha=0.8, linewidth=1.5)
                ax.grid(True, alpha=0.4, axis='y', linestyle='-', linewidth=0.5)
                ax.set_ylabel('Impacto (%)', fontsize=11, fontweight='500', color='#34495e')

                # Valores nas barras
                for bar, valor in zip(bars, valores):
                    height = bar.get_height()
                    if valor >= 0:
                        text_y = height + (margem * 0.1)
                        va = 'bottom'
                    else:
                        text_y = height - (margem * 0.1)
                        va = 'top'

                    ax.text(bar.get_x() + bar.get_width()/2., text_y,
                           f'{valor:+.1f}%', ha='center', va=va,
                           fontweight='bold', fontsize=10, color='#2c3e50')

                ax.tick_params(axis='x', rotation=20, labelsize=10, colors='#34495e')
                ax.tick_params(axis='y', labelsize=10, colors='#34495e')

                ax.set_facecolor('#fafbfc')
                for spine in ax.spines.values():
                    spine.set_color('#bdc3c7')
                    spine.set_linewidth(1)
            else:
                ax.text(0.5, 0.5, 'Dados setoriais\nnão disponíveis',
                       transform=ax.transAxes, ha='center', va='center',
                       fontsize=12, style='italic', color='#7f8c8d',
                       bbox=dict(boxstyle='round,pad=0.5', facecolor='#ecf0f1', alpha=0.8))
                ax.set_title('Impacto por Setor Econômico',
                            fontsize=14, fontweight='bold', pad=15, color='#2c3e50')

        except Exception as e:
            ax.text(0.5, 0.5, f'Erro no gráfico setorial:\n{str(e)}',
                   transform=ax.transAxes, ha='center', va='center',
                   bbox=dict(boxstyle='round,pad=0.5', facecolor='#e74c3c', alpha=0.8),
                   fontsize=10, color='white', fontweight='bold')

    def limpar_valores(self):
        """Limpa apenas os valores dos inputs"""
        for input_widget in self.inputs.values():
            input_widget.value = 0

    def limpar(self, b):
        """Limpa todos os campos e abas"""
        self.limpar_valores()
        self.dropdown_cenarios.value = 'neutro'
        self.dropdown_setorial.value = 'neutro'
        with self.aba_resumo:
            clear_output()
        with self.aba_graficos:
            clear_output()

    def mostrar(self):
        """Exibe a interface completa"""
        display(self.interface)


# FUNÇÃO PRINCIPAL CORRIGIDA - SEM IMPORT PROBLEMÁTICO
def criar_dashboard_com_abas_corrigido(df_fundo, modelo_ibov, modelo_dolar, modelo_cupom, modelo_selic):
    """
    Cria o dashboard completo com abas funcionando - VERSÃO CORRIGIDA

    Usage:
    dashboard = criar_dashboard_com_abas_corrigido(df, modelo_ibov, modelo_dolar, modelo_cupom, modelo_selic)
    dashboard.mostrar()
    """
    print("🚀 Inicializando Dashboard com Abas - VERSÃO CORRIGIDA...")

    # Usar as classes que já foram definidas nas seções anteriores (5.1 a 5.5)
    # Não precisa de import - as classes já estão na memória
    motor_completo = MotorRiscoCompleto(df_fundo, modelo_ibov, modelo_dolar, modelo_cupom, modelo_selic)

    # Criar interface com abas
    dashboard = DashboardComAbas(motor_completo)

    print("✅ Dashboard com Abas pronto para uso!")
    print("🎯 Funcionalidades:")
    print("   • ✅ Aba 1: Resumo Executivo")
    print("   • ✅ Aba 2: Gráficos (Drawdown, Benchmarks, Setorial)")
    print("   • ✅ Cenários Históricos (Impeachment, COVID, etc.)")
    print("   • ✅ Stress Setorial (Financeiro, Commodities, etc.)")
    print("   • ✅ Checkboxes para selecionar gráficos")

    return dashboard


# ===================================
# 🚀 EXECUÇÃO
# ===================================
print("="*60)
print("✅ CÓDIGO CORRIGIDO E PRONTO!")
print("="*60)
print()
print("📋 Para executar:")
print("1. Execute esta célula")
print("2. Execute o comando abaixo:")
print()
print("dashboard = criar_dashboard_com_abas_corrigido(df, modelo_ibov, modelo_dolar, modelo_cupom, modelo_selic)")
print("dashboard.mostrar()")
print()
print("🎯 Layout corrigido:")
print("   • 📊 Quadro 1: Semáforo + Resumo Executivo")
print("   • 📋 Quadro 2: Métricas Principais + Breakdown")
print("   • 📉 Quadro 3: Drawdown (separado)")
print("   • 🏆 Quadro 4: Benchmarks (separado)")
print()
print("✅ Erros de indentação corrigidos!")

✅ CÓDIGO CORRIGIDO E PRONTO!

📋 Para executar:
1. Execute esta célula
2. Execute o comando abaixo:

dashboard = criar_dashboard_com_abas_corrigido(df, modelo_ibov, modelo_dolar, modelo_cupom, modelo_selic)
dashboard.mostrar()

🎯 Layout corrigido:
   • 📊 Quadro 1: Semáforo + Resumo Executivo
   • 📋 Quadro 2: Métricas Principais + Breakdown
   • 📉 Quadro 3: Drawdown (separado)
   • 🏆 Quadro 4: Benchmarks (separado)

✅ Erros de indentação corrigidos!


## 5.2. EXECUTAR DASHBOARD CORRIGIDO

In [33]:
dashboard = criar_dashboard_com_abas_corrigido(df, modelo_ibov, modelo_dolar, modelo_cupom, modelo_selic)
dashboard.mostrar()

/tmp/ipython-input-22-4287597009.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'ibovespa': modelo_ibov.params[1],
/tmp/ipython-input-22-4287597009.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'dolar': modelo_dolar.params[1],
/tmp/ipython-input-22-4287597009.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  'cupom': modelo_cupom.params[1],
/tmp/ipython-input-22-4287597009.py:29: FutureWarning: Series.__getitem__ tre

🚀 Inicializando Dashboard com Abas - VERSÃO CORRIGIDA...
✅ Motor de Stress Test inicializado!
📊 Cota atual: R$ 816.2130
📈 Volatilidade 21d: 9.77%
✅ Benchmarks carregados com sucesso!
🚀 Motor de Risco Completo inicializado!
📊 Componentes disponíveis:
   • Análise de Drawdown
   • Stress Test Setorial
   • Comparação com Benchmarks
   • Dashboard Interativo 2.0
✅ Dashboard com Abas pronto para uso!
🎯 Funcionalidades:
   • ✅ Aba 1: Resumo Executivo
   • ✅ Aba 2: Gráficos (Drawdown, Benchmarks, Setorial)
   • ✅ Cenários Históricos (Impeachment, COVID, etc.)
   • ✅ Stress Setorial (Financeiro, Commodities, etc.)
   • ✅ Checkboxes para selecionar gráficos
